In [1]:
!pip install optuna grande entmax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensor

In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, make_scorer
from scipy.stats import shapiro
import optuna
from ncart import NCARClassifier

In [3]:
# This is a function that loads training and testing data for 10-fold cross validation
def prepare_train_test_dataset(datasets, test_idx):
  test_df = datasets[test_idx]
  train_df = pd.concat([datasets[i] for i in range(len(datasets)) if i != test_idx])

  X_train = train_df.iloc[:, 1:-1].values
  X_test = test_df.iloc[:, 1:-1].values
  y_train = train_df.iloc[:, -1].values
  y_test = test_df.iloc[:, -1].values

  return X_train, X_test, y_train, y_test

In [4]:
import torch

def train_with_base_ncart(datasets, key):
  print("Currently training NCART model with dataset key:", key)

  # Create a Pandas DataFrame to store all experiment results
  results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'MCC'])

  # Perform 10-fold cross validation
  dataset_list = datasets[key]

  for i in range(10):
    X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, i)

    # Train the model
    model = NCARClassifier()
    model.fit(X=X_train, y=y_train, X_val=X_test, y_val=y_test)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    print("Fold", i+1, "Accuracy:", accuracy, "MCC:", mcc)
    results_df.loc[i] = [i+1, accuracy, mcc]

  return results_df

In [5]:
def create_objective_for_ncart(X_train, X_test, y_train, y_test):
  def objective(trial):
      # Suggest hyperparameters for GRANDE
      params = {
          "n_trees": trial.suggest_int("n_trees", 8, 128, step=8),
          "n_selected": trial.suggest_int("n_selected", 4, 32, step=4),
          "n_layers": trial.suggest_int("n_layers", 1, 8),
          "mask_type": trial.suggest_categorical("mask_type", ["sparsemax", "entmax"]),
      }

      base_model = NCARClassifier(
          n_trees=params["n_trees"],
          n_selected=params["n_selected"],
          n_layers=params["n_layers"],
          mask_type=params["mask_type"],
      )
      base_model.fit(X=X_train, y=y_train, X_val=X_test, y_val=y_test)

      y_pred = base_model.predict(X_test)
      mcc = matthews_corrcoef(y_test, y_pred)

      # Return the accuracy
      return mcc

  return objective

In [6]:
def train_with_best_hyperparameters(datasets, key):
  print("Currently training NCART model with dataset key:", key)

  # Create a Pandas DataFrame to store all experiment results
  results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'MCC'])

  # Perform 10-fold cross validation
  dataset_list = datasets[key]

  # Use the first data as the test for the HPO
  X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, 0)

  objective_function = create_objective_for_ncart(X_train, X_test, y_train, y_test)

  study = optuna.create_study(direction='maximize')
  study.optimize(objective_function, n_trials=15, gc_after_trial=True)

  best_params = study.best_params
  print("Best hyperparameters:", best_params)

  # Use best hyperparams to conduct a 10-fold cross validation
  for i in range(10):
    X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, i)

    # Train the model
    model = NCARClassifier(
        n_trees=best_params["n_trees"],
        n_selected=best_params["n_selected"],
        n_layers=best_params["n_layers"],
        mask_type=best_params["mask_type"],
    )
    model.fit(X=X_train, y=y_train, X_val=X_test, y_val=y_test)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    print("Fold", i+1, "Accuracy:", accuracy, "MCC:", mcc)
    results_df.loc[i] = [i+1, accuracy, mcc]

  return results_df

In [7]:
# Load the validation datasets

datasets = {}

# All

IX_all_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_1.csv')
IX_all_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_2.csv')
IX_all_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_3.csv')
IX_all_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_4.csv')
IX_all_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_5.csv')
IX_all_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_6.csv')
IX_all_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_7.csv')
IX_all_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_8.csv')
IX_all_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_9.csv')
IX_all_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_10.csv')
datasets['IX_all_nopcc'] = [IX_all_nopcc_1, IX_all_nopcc_2, IX_all_nopcc_3, IX_all_nopcc_4, IX_all_nopcc_5,
                                       IX_all_nopcc_6, IX_all_nopcc_7, IX_all_nopcc_8, IX_all_nopcc_9, IX_all_nopcc_10]

IX_all_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_1.csv')
IX_all_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_2.csv')
IX_all_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_3.csv')
IX_all_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_4.csv')
IX_all_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_5.csv')
IX_all_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_6.csv')
IX_all_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_7.csv')
IX_all_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_8.csv')
IX_all_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_9.csv')
IX_all_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_10.csv')
datasets['IX_all_pcc95'] = [IX_all_pcc95_1, IX_all_pcc95_2, IX_all_pcc95_3, IX_all_pcc95_4, IX_all_pcc95_5,
                                       IX_all_pcc95_6, IX_all_pcc95_7, IX_all_pcc95_8, IX_all_pcc95_9, IX_all_pcc95_10]

IX_all_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_1.csv')
IX_all_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_2.csv')
IX_all_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_3.csv')
IX_all_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_4.csv')
IX_all_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_5.csv')
IX_all_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_6.csv')
IX_all_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_7.csv')
IX_all_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_8.csv')
IX_all_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_9.csv')
IX_all_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_10.csv')
datasets['IX_all_pcc75'] = [IX_all_pcc75_1, IX_all_pcc75_2, IX_all_pcc75_3, IX_all_pcc75_4, IX_all_pcc75_5,
                                       IX_all_pcc75_6, IX_all_pcc75_7, IX_all_pcc75_8, IX_all_pcc75_9, IX_all_pcc75_10]

# 500

IX_500_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_1.csv')
IX_500_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_2.csv')
IX_500_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_3.csv')
IX_500_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_4.csv')
IX_500_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_5.csv')
IX_500_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_6.csv')
IX_500_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_7.csv')
IX_500_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_8.csv')
IX_500_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_9.csv')
IX_500_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_10.csv')
datasets['IX_500_nopcc'] = [IX_500_nopcc_1, IX_500_nopcc_2, IX_500_nopcc_3, IX_500_nopcc_4, IX_500_nopcc_5,
                                       IX_500_nopcc_6, IX_500_nopcc_7, IX_500_nopcc_8, IX_500_nopcc_9, IX_500_nopcc_10]

IX_500_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_1.csv')
IX_500_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_2.csv')
IX_500_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_3.csv')
IX_500_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_4.csv')
IX_500_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_5.csv')
IX_500_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_6.csv')
IX_500_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_7.csv')
IX_500_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_8.csv')
IX_500_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_9.csv')
IX_500_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_10.csv')
datasets['IX_500_pcc95'] = [IX_500_pcc95_1, IX_500_pcc95_2, IX_500_pcc95_3, IX_500_pcc95_4, IX_500_pcc95_5,
                                       IX_500_pcc95_6, IX_500_pcc95_7, IX_500_pcc95_8, IX_500_pcc95_9, IX_500_pcc95_10]

IX_500_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_1.csv')
IX_500_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_2.csv')
IX_500_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_3.csv')
IX_500_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_4.csv')
IX_500_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_5.csv')
IX_500_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_6.csv')
IX_500_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_7.csv')
IX_500_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_8.csv')
IX_500_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_9.csv')
IX_500_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_10.csv')
datasets['IX_500_pcc75'] = [IX_500_pcc75_1, IX_500_pcc75_2, IX_500_pcc75_3, IX_500_pcc75_4, IX_500_pcc75_5,
                                       IX_500_pcc75_6, IX_500_pcc75_7, IX_500_pcc75_8, IX_500_pcc75_9, IX_500_pcc75_10]

# 400

IX_400_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_1.csv')
IX_400_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_2.csv')
IX_400_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_3.csv')
IX_400_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_4.csv')
IX_400_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_5.csv')
IX_400_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_6.csv')
IX_400_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_7.csv')
IX_400_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_8.csv')
IX_400_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_9.csv')
IX_400_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_10.csv')
datasets['IX_400_nopcc'] = [IX_400_nopcc_1, IX_400_nopcc_2, IX_400_nopcc_3, IX_400_nopcc_4, IX_400_nopcc_5,
                                       IX_400_nopcc_6, IX_400_nopcc_7, IX_400_nopcc_8, IX_400_nopcc_9, IX_400_nopcc_10]

IX_400_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_1.csv')
IX_400_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_2.csv')
IX_400_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_3.csv')
IX_400_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_4.csv')
IX_400_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_5.csv')
IX_400_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_6.csv')
IX_400_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_7.csv')
IX_400_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_8.csv')
IX_400_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_9.csv')
IX_400_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_10.csv')
datasets['IX_400_pcc95'] = [IX_400_pcc95_1, IX_400_pcc95_2, IX_400_pcc95_3, IX_400_pcc95_4, IX_400_pcc95_5,
                                       IX_400_pcc95_6, IX_400_pcc95_7, IX_400_pcc95_8, IX_400_pcc95_9, IX_400_pcc95_10]

IX_400_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_1.csv')
IX_400_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_2.csv')
IX_400_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_3.csv')
IX_400_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_4.csv')
IX_400_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_5.csv')
IX_400_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_6.csv')
IX_400_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_7.csv')
IX_400_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_8.csv')
IX_400_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_9.csv')
IX_400_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_10.csv')
datasets['IX_400_pcc75'] = [IX_400_pcc75_1, IX_400_pcc75_2, IX_400_pcc75_3, IX_400_pcc75_4, IX_400_pcc75_5,
                                       IX_400_pcc75_6, IX_400_pcc75_7, IX_400_pcc75_8, IX_400_pcc75_9, IX_400_pcc75_10]

# 300

IX_300_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_1.csv')
IX_300_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_2.csv')
IX_300_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_3.csv')
IX_300_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_4.csv')
IX_300_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_5.csv')
IX_300_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_6.csv')
IX_300_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_7.csv')
IX_300_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_8.csv')
IX_300_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_9.csv')
IX_300_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_10.csv')
datasets['IX_300_nopcc'] = [IX_300_nopcc_1, IX_300_nopcc_2, IX_300_nopcc_3, IX_300_nopcc_4, IX_300_nopcc_5,
                                       IX_300_nopcc_6, IX_300_nopcc_7, IX_300_nopcc_8, IX_300_nopcc_9, IX_300_nopcc_10]

IX_300_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_1.csv')
IX_300_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_2.csv')
IX_300_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_3.csv')
IX_300_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_4.csv')
IX_300_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_5.csv')
IX_300_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_6.csv')
IX_300_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_7.csv')
IX_300_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_8.csv')
IX_300_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_9.csv')
IX_300_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_10.csv')
datasets['IX_300_pcc95'] = [IX_300_pcc95_1, IX_300_pcc95_2, IX_300_pcc95_3, IX_300_pcc95_4, IX_300_pcc95_5,
                                       IX_300_pcc95_6, IX_300_pcc95_7, IX_300_pcc95_8, IX_300_pcc95_9, IX_300_pcc95_10]

IX_300_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_1.csv')
IX_300_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_2.csv')
IX_300_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_3.csv')
IX_300_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_4.csv')
IX_300_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_5.csv')
IX_300_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_6.csv')
IX_300_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_7.csv')
IX_300_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_8.csv')
IX_300_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_9.csv')
IX_300_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_10.csv')
datasets['IX_300_pcc75'] = [IX_300_pcc75_1, IX_300_pcc75_2, IX_300_pcc75_3, IX_300_pcc75_4, IX_300_pcc75_5,
                                       IX_300_pcc75_6, IX_300_pcc75_7, IX_300_pcc75_8, IX_300_pcc75_9, IX_300_pcc75_10]


In [8]:
def view_results_df(results_df, key):
    print("10-fold cross validation results for results_df with key:", key)
    for i in range(len(results_df)):
        # Format accuracy and MCC to 3 decimal places
        accuracy = f"{results_df['Accuracy'][i]:.3f}"
        mcc = f"{results_df['MCC'][i]:.3f}"
        print(f"Fold {i}, Accuracy: {accuracy}, MCC: {mcc}")


In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
import time
start_time = time.time()
IX_all_nopcc_results_df = train_with_base_ncart(datasets, 'IX_all_nopcc')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

Currently training NCART model with dataset key: IX_all_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.797144 |  Val Loss 0.782395
Epoch 1: Train Loss 0.766021 |  Val Loss 0.763669
Epoch 2: Train Loss 0.737863 |  Val Loss 0.748742
Epoch 3: Train Loss 0.731428 |  Val Loss 0.735796
Epoch 4: Train Loss 0.727644 |  Val Loss 0.724807
Epoch 5: Train Loss 0.703506 |  Val Loss 0.714791
Epoch 6: Train Loss 0.695085 |  Val Loss 0.704562
Epoch 7: Train Loss 0.679532 |  Val Loss 0.695131
Epoch 8: Train Loss 0.668359 |  Val Loss 0.686888
Epoch 9: Train Loss 0.668579 |  Val Loss 0.680308
Epoch 10: Train Loss 0.649103 |  Val Loss 0.673257
Epoch 11: Train Loss 0.634753 |  Val Loss 0.666645
Epoch 12: Train Loss 0.636675 |  Val Loss 0.660432
Epoch 13: Train Loss 0.633180 |  Val Loss 0.655180
Epoch 14: Train Loss 0.618931 |  Val Loss 0.650562
Epoch 15: Train Loss 0.613652 |  Val Loss 0.645213
Epoch 16: Train Loss 0.611577 |  Val Loss 0.639380
Epoch 17: Train Loss 0.607401 |  Val Los

In [11]:
start_time = time.time()
IX_all_pcc95_results_df = train_with_base_ncart(datasets, 'IX_all_pcc95')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

Currently training NCART model with dataset key: IX_all_pcc95
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.723284 |  Val Loss 0.709829
Epoch 1: Train Loss 0.709726 |  Val Loss 0.699752
Epoch 2: Train Loss 0.699887 |  Val Loss 0.690924
Epoch 3: Train Loss 0.687214 |  Val Loss 0.683553
Epoch 4: Train Loss 0.669640 |  Val Loss 0.677323
Epoch 5: Train Loss 0.672077 |  Val Loss 0.671957
Epoch 6: Train Loss 0.665406 |  Val Loss 0.666866
Epoch 7: Train Loss 0.655922 |  Val Loss 0.662604
Epoch 8: Train Loss 0.656639 |  Val Loss 0.658976
Epoch 9: Train Loss 0.644050 |  Val Loss 0.655785
Epoch 10: Train Loss 0.638113 |  Val Loss 0.652845
Epoch 11: Train Loss 0.630673 |  Val Loss 0.649941
Epoch 12: Train Loss 0.630100 |  Val Loss 0.646937
Epoch 13: Train Loss 0.623117 |  Val Loss 0.643981
Epoch 14: Train Loss 0.614863 |  Val Loss 0.641258
Epoch 15: Train Loss 0.614760 |  Val Loss 0.638303
Epoch 16: Train Loss 0.604654 |  Val Loss 0.634976
Epoch 17: Train Loss 0.602301 |  Val Los

In [12]:
start_time = time.time()
IX_all_pcc75_results_df = train_with_base_ncart(datasets, 'IX_all_pcc75')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Currently training NCART model with dataset key: IX_all_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.788781 |  Val Loss 0.769717
Epoch 1: Train Loss 0.777817 |  Val Loss 0.755265
Epoch 2: Train Loss 0.766151 |  Val Loss 0.741453
Epoch 3: Train Loss 0.735680 |  Val Loss 0.728296
Epoch 4: Train Loss 0.710932 |  Val Loss 0.716697
Epoch 5: Train Loss 0.711158 |  Val Loss 0.706479
Epoch 6: Train Loss 0.699689 |  Val Loss 0.697399
Epoch 7: Train Loss 0.683264 |  Val Loss 0.689593
Epoch 8: Train Loss 0.680264 |  Val Loss 0.682553
Epoch 9: Train Loss 0.661320 |  Val Loss 0.676624
Epoch 10: Train Loss 0.655494 |  Val Loss 0.671686
Epoch 11: Train Loss 0.651890 |  Val Loss 0.666914
Epoch 12: Train Loss 0.650035 |  Val Loss 0.662365
Epoch 13: Train Loss 0.640072 |  Val Loss 0.658163
Epoch 14: Train Loss 0.630789 |  Val Loss 0.654087
Epoch 15: Train Loss 0.633102 |  Val Loss 0.650261
Epoch 16: Train Loss 0.625061 |  Val Loss 0.646744
Epoch 17: Train Loss 0.620332 |  Val Los

In [13]:
start_time = time.time()
IX_500_nopcc_results_df = train_with_base_ncart(datasets, 'IX_500_nopcc')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

Currently training NCART model with dataset key: IX_500_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.735673 |  Val Loss 0.732116
Epoch 1: Train Loss 0.718311 |  Val Loss 0.718811
Epoch 2: Train Loss 0.704225 |  Val Loss 0.706985
Epoch 3: Train Loss 0.691807 |  Val Loss 0.697373
Epoch 4: Train Loss 0.681115 |  Val Loss 0.689158
Epoch 5: Train Loss 0.672396 |  Val Loss 0.680933
Epoch 6: Train Loss 0.664791 |  Val Loss 0.674005
Epoch 7: Train Loss 0.657025 |  Val Loss 0.668058
Epoch 8: Train Loss 0.650299 |  Val Loss 0.661688
Epoch 9: Train Loss 0.644211 |  Val Loss 0.657642
Epoch 10: Train Loss 0.638324 |  Val Loss 0.653034
Epoch 11: Train Loss 0.632295 |  Val Loss 0.648003
Epoch 12: Train Loss 0.626442 |  Val Loss 0.644976
Epoch 13: Train Loss 0.620914 |  Val Loss 0.640813
Epoch 14: Train Loss 0.615763 |  Val Loss 0.637252
Epoch 15: Train Loss 0.610837 |  Val Loss 0.634609
Epoch 16: Train Loss 0.605955 |  Val Loss 0.631667
Epoch 17: Train Loss 0.601476 |  Val Los

In [14]:
start_time = time.time()
IX_500_pcc95_results_df = train_with_base_ncart(datasets, 'IX_500_pcc95')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

Currently training NCART model with dataset key: IX_500_pcc95
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.702493 |  Val Loss 0.695750
Epoch 1: Train Loss 0.697538 |  Val Loss 0.692329
Epoch 2: Train Loss 0.693235 |  Val Loss 0.689223
Epoch 3: Train Loss 0.689466 |  Val Loss 0.686599
Epoch 4: Train Loss 0.686031 |  Val Loss 0.684063
Epoch 5: Train Loss 0.682858 |  Val Loss 0.681781
Epoch 6: Train Loss 0.679877 |  Val Loss 0.679520
Epoch 7: Train Loss 0.676918 |  Val Loss 0.677481
Epoch 8: Train Loss 0.674084 |  Val Loss 0.675526
Epoch 9: Train Loss 0.671254 |  Val Loss 0.673484
Epoch 10: Train Loss 0.668241 |  Val Loss 0.670869
Epoch 11: Train Loss 0.665279 |  Val Loss 0.668671
Epoch 12: Train Loss 0.662571 |  Val Loss 0.666585
Epoch 13: Train Loss 0.660002 |  Val Loss 0.664790
Epoch 14: Train Loss 0.657604 |  Val Loss 0.662946
Epoch 15: Train Loss 0.655173 |  Val Loss 0.660726
Epoch 16: Train Loss 0.652744 |  Val Loss 0.658588
Epoch 17: Train Loss 0.650327 |  Val Los

In [15]:
start_time = time.time()
IX_500_pcc75_results_df = train_with_base_ncart(datasets, 'IX_500_pcc75')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

Currently training NCART model with dataset key: IX_500_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.744274 |  Val Loss 0.737364
Epoch 1: Train Loss 0.737197 |  Val Loss 0.730171
Epoch 2: Train Loss 0.731380 |  Val Loss 0.725006
Epoch 3: Train Loss 0.726354 |  Val Loss 0.720120
Epoch 4: Train Loss 0.721793 |  Val Loss 0.716035
Epoch 5: Train Loss 0.717319 |  Val Loss 0.711927
Epoch 6: Train Loss 0.713105 |  Val Loss 0.708068
Epoch 7: Train Loss 0.709057 |  Val Loss 0.704333
Epoch 8: Train Loss 0.705120 |  Val Loss 0.700872
Epoch 9: Train Loss 0.701296 |  Val Loss 0.697655
Epoch 10: Train Loss 0.697473 |  Val Loss 0.694600
Epoch 11: Train Loss 0.693692 |  Val Loss 0.691710
Epoch 12: Train Loss 0.690027 |  Val Loss 0.688718
Epoch 13: Train Loss 0.686566 |  Val Loss 0.685587
Epoch 14: Train Loss 0.683118 |  Val Loss 0.682607
Epoch 15: Train Loss 0.679763 |  Val Loss 0.679909
Epoch 16: Train Loss 0.676525 |  Val Loss 0.677441
Epoch 17: Train Loss 0.673482 |  Val Los

In [16]:
start_time = time.time()
IX_400_nopcc_results_df = train_with_base_ncart(datasets, 'IX_400_nopcc')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Currently training NCART model with dataset key: IX_400_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.742264 |  Val Loss 0.722662
Epoch 1: Train Loss 0.722347 |  Val Loss 0.709810
Epoch 2: Train Loss 0.706199 |  Val Loss 0.697994
Epoch 3: Train Loss 0.692369 |  Val Loss 0.686133
Epoch 4: Train Loss 0.680934 |  Val Loss 0.676457
Epoch 5: Train Loss 0.671144 |  Val Loss 0.668373
Epoch 6: Train Loss 0.662586 |  Val Loss 0.661007
Epoch 7: Train Loss 0.654946 |  Val Loss 0.654536
Epoch 8: Train Loss 0.647556 |  Val Loss 0.649380
Epoch 9: Train Loss 0.640573 |  Val Loss 0.643964
Epoch 10: Train Loss 0.633961 |  Val Loss 0.638919
Epoch 11: Train Loss 0.627779 |  Val Loss 0.634415
Epoch 12: Train Loss 0.621864 |  Val Loss 0.630780
Epoch 13: Train Loss 0.616252 |  Val Loss 0.627496
Epoch 14: Train Loss 0.610800 |  Val Loss 0.624647
Epoch 15: Train Loss 0.605612 |  Val Loss 0.621388
Epoch 16: Train Loss 0.600664 |  Val Loss 0.618684
Epoch 17: Train Loss 0.595887 |  Val Los

In [17]:
print("\n---------------------------------------------------------------------\n")
start_time = time.time()
IX_400_pcc95_results_df = train_with_base_ncart(datasets, 'IX_400_pcc95')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")


---------------------------------------------------------------------

Currently training NCART model with dataset key: IX_400_pcc95
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.703224 |  Val Loss 0.689034
Epoch 1: Train Loss 0.697817 |  Val Loss 0.684157
Epoch 2: Train Loss 0.692969 |  Val Loss 0.680325
Epoch 3: Train Loss 0.688750 |  Val Loss 0.676724
Epoch 4: Train Loss 0.685077 |  Val Loss 0.673494
Epoch 5: Train Loss 0.681647 |  Val Loss 0.670829
Epoch 6: Train Loss 0.678568 |  Val Loss 0.668132
Epoch 7: Train Loss 0.675484 |  Val Loss 0.665484
Epoch 8: Train Loss 0.672571 |  Val Loss 0.662760
Epoch 9: Train Loss 0.669781 |  Val Loss 0.660099
Epoch 10: Train Loss 0.667005 |  Val Loss 0.657150
Epoch 11: Train Loss 0.664282 |  Val Loss 0.654329
Epoch 12: Train Loss 0.661574 |  Val Loss 0.651563
Epoch 13: Train Loss 0.658852 |  Val Loss 0.648990
Epoch 14: Train Loss 0.656165 |  Val Loss 0.646281
Epoch 15: Train Loss 0.653494 |  Val Loss 0.643474
Epoch 16: Train Los

In [18]:
start_time = time.time()
IX_400_pcc75_results_df = train_with_base_ncart(datasets, 'IX_400_pcc75')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

Currently training NCART model with dataset key: IX_400_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.745025 |  Val Loss 0.734056
Epoch 1: Train Loss 0.737082 |  Val Loss 0.727146
Epoch 2: Train Loss 0.730660 |  Val Loss 0.722379
Epoch 3: Train Loss 0.725324 |  Val Loss 0.718119
Epoch 4: Train Loss 0.720195 |  Val Loss 0.714345
Epoch 5: Train Loss 0.715366 |  Val Loss 0.710484
Epoch 6: Train Loss 0.710712 |  Val Loss 0.707390
Epoch 7: Train Loss 0.706105 |  Val Loss 0.704081
Epoch 8: Train Loss 0.701843 |  Val Loss 0.700929
Epoch 9: Train Loss 0.697639 |  Val Loss 0.697670
Epoch 10: Train Loss 0.693554 |  Val Loss 0.694211
Epoch 11: Train Loss 0.689571 |  Val Loss 0.690735
Epoch 12: Train Loss 0.685695 |  Val Loss 0.687095
Epoch 13: Train Loss 0.681887 |  Val Loss 0.683600
Epoch 14: Train Loss 0.678191 |  Val Loss 0.680092
Epoch 15: Train Loss 0.674575 |  Val Loss 0.677013
Epoch 16: Train Loss 0.671046 |  Val Loss 0.674086
Epoch 17: Train Loss 0.667551 |  Val Los

In [19]:
start_time = time.time()
IX_300_nopcc_results_df = train_with_base_ncart(datasets, 'IX_300_nopcc')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

Currently training NCART model with dataset key: IX_300_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.746438 |  Val Loss 0.733478
Epoch 1: Train Loss 0.731094 |  Val Loss 0.720818
Epoch 2: Train Loss 0.718982 |  Val Loss 0.708636
Epoch 3: Train Loss 0.707700 |  Val Loss 0.697724
Epoch 4: Train Loss 0.697513 |  Val Loss 0.687377
Epoch 5: Train Loss 0.687311 |  Val Loss 0.677394
Epoch 6: Train Loss 0.677143 |  Val Loss 0.667709
Epoch 7: Train Loss 0.667865 |  Val Loss 0.659508
Epoch 8: Train Loss 0.658552 |  Val Loss 0.652503
Epoch 9: Train Loss 0.649367 |  Val Loss 0.645849
Epoch 10: Train Loss 0.640665 |  Val Loss 0.639053
Epoch 11: Train Loss 0.632422 |  Val Loss 0.632207
Epoch 12: Train Loss 0.624449 |  Val Loss 0.625214
Epoch 13: Train Loss 0.616973 |  Val Loss 0.619291
Epoch 14: Train Loss 0.609866 |  Val Loss 0.613648
Epoch 15: Train Loss 0.602730 |  Val Loss 0.608483
Epoch 16: Train Loss 0.595737 |  Val Loss 0.603106
Epoch 17: Train Loss 0.588973 |  Val Los

In [20]:
start_time = time.time()
IX_300_pcc95_results_df = train_with_base_ncart(datasets, 'IX_300_pcc95')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

Currently training NCART model with dataset key: IX_300_pcc95
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.759411 |  Val Loss 0.759865
Epoch 1: Train Loss 0.744183 |  Val Loss 0.745975
Epoch 2: Train Loss 0.731682 |  Val Loss 0.734828
Epoch 3: Train Loss 0.720035 |  Val Loss 0.724880
Epoch 4: Train Loss 0.708965 |  Val Loss 0.716164
Epoch 5: Train Loss 0.698780 |  Val Loss 0.707457
Epoch 6: Train Loss 0.689445 |  Val Loss 0.698796
Epoch 7: Train Loss 0.680871 |  Val Loss 0.690848
Epoch 8: Train Loss 0.672839 |  Val Loss 0.683020
Epoch 9: Train Loss 0.665222 |  Val Loss 0.675971
Epoch 10: Train Loss 0.657842 |  Val Loss 0.669996
Epoch 11: Train Loss 0.650879 |  Val Loss 0.664038
Epoch 12: Train Loss 0.644436 |  Val Loss 0.658341
Epoch 13: Train Loss 0.638198 |  Val Loss 0.652605
Epoch 14: Train Loss 0.632115 |  Val Loss 0.647307
Epoch 15: Train Loss 0.626138 |  Val Loss 0.641983
Epoch 16: Train Loss 0.620648 |  Val Loss 0.637154
Epoch 17: Train Loss 0.615429 |  Val Los

In [21]:
start_time = time.time()
IX_300_pcc75_results_df = train_with_base_ncart(datasets, 'IX_300_pcc75')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Currently training NCART model with dataset key: IX_300_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.695778 |  Val Loss 0.676493
Epoch 1: Train Loss 0.690197 |  Val Loss 0.674382
Epoch 2: Train Loss 0.685125 |  Val Loss 0.671639
Epoch 3: Train Loss 0.680748 |  Val Loss 0.668870
Epoch 4: Train Loss 0.676785 |  Val Loss 0.665555
Epoch 5: Train Loss 0.673023 |  Val Loss 0.662656
Epoch 6: Train Loss 0.669626 |  Val Loss 0.660098
Epoch 7: Train Loss 0.666362 |  Val Loss 0.657480
Epoch 8: Train Loss 0.663189 |  Val Loss 0.655373
Epoch 9: Train Loss 0.660223 |  Val Loss 0.653368
Epoch 10: Train Loss 0.657462 |  Val Loss 0.651383
Epoch 11: Train Loss 0.654826 |  Val Loss 0.649527
Epoch 12: Train Loss 0.652416 |  Val Loss 0.647887
Epoch 13: Train Loss 0.650056 |  Val Loss 0.646413
Epoch 14: Train Loss 0.647703 |  Val Loss 0.644813
Epoch 15: Train Loss 0.645335 |  Val Loss 0.642802
Epoch 16: Train Loss 0.642852 |  Val Loss 0.640538
Epoch 17: Train Loss 0.640297 |  Val Los

In [22]:
print("Base Model:")
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_nopcc_results_df, 'IX_all_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_pcc95_results_df, 'IX_all_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_nopcc_results_df, 'IX_all_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_nopcc_results_df, 'IX_500_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_pcc95_results_df, 'IX_500_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_nopcc_results_df, 'IX_500_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_nopcc_results_df, 'IX_400_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_pcc95_results_df, 'IX_400_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_nopcc_results_df, 'IX_400_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_nopcc_results_df, 'IX_300_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_pcc95_results_df, 'IX_300_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_nopcc_results_df, 'IX_300_pcc75')
print('\n---------------------------------------------------------------------\n')


Base Model:

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: IX_all_nopcc
Fold 0, Accuracy: 0.841, MCC: 0.683
Fold 1, Accuracy: 0.855, MCC: 0.713
Fold 2, Accuracy: 0.828, MCC: 0.662
Fold 3, Accuracy: 0.833, MCC: 0.668
Fold 4, Accuracy: 0.812, MCC: 0.627
Fold 5, Accuracy: 0.833, MCC: 0.684
Fold 6, Accuracy: 0.819, MCC: 0.643
Fold 7, Accuracy: 0.722, MCC: 0.463
Fold 8, Accuracy: 0.764, MCC: 0.533
Fold 9, Accuracy: 0.833, MCC: 0.669

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: IX_all_pcc95
Fold 0, Accuracy: 0.814, MCC: 0.628
Fold 1, Accuracy: 0.800, MCC: 0.614
Fold 2, Accuracy: 0.807, MCC: 0.645
Fold 3, Accuracy: 0.812, MCC: 0.625
Fold 4, Accuracy: 0.771, MCC: 0.551
Fold 5, Accuracy: 0.868, MCC: 0.737
Fold 6, Accuracy: 0.764, MCC: 0.528
Fold 7, Accuracy: 0.819, MCC: 0.641
Fold 8, Accuracy: 0.806, MCC: 0.613
Fold 9, Accuracy: 

In [23]:
start_time = time.time()
IX_all_nopcc_results_df = train_with_best_hyperparameters(datasets, 'IX_all_nopcc')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

[I 2025-01-04 15:22:12,027] A new study created in memory with name: no-name-da59b7ed-d544-4b11-97f6-05eefeb71ecd


Currently training NCART model with dataset key: IX_all_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.736663 |  Val Loss 0.733041
Epoch 1: Train Loss 0.722432 |  Val Loss 0.725322
Epoch 2: Train Loss 0.716298 |  Val Loss 0.722196
Epoch 3: Train Loss 0.713378 |  Val Loss 0.721853
Epoch 4: Train Loss 0.715034 |  Val Loss 0.721452
Epoch 5: Train Loss 0.713620 |  Val Loss 0.719647
Epoch 6: Train Loss 0.709681 |  Val Loss 0.717158
Epoch 7: Train Loss 0.705717 |  Val Loss 0.714469
Epoch 8: Train Loss 0.703474 |  Val Loss 0.712125
Epoch 9: Train Loss 0.704347 |  Val Loss 0.710312
Epoch 10: Train Loss 0.698020 |  Val Loss 0.708994
Epoch 11: Train Loss 0.694366 |  Val Loss 0.707895
Epoch 12: Train Loss 0.700327 |  Val Loss 0.706553
Epoch 13: Train Loss 0.692489 |  Val Loss 0.705027
Epoch 14: Train Loss 0.689588 |  Val Loss 0.703477
Epoch 15: Train Loss 0.692287 |  Val Loss 0.701973
Epoch 16: Train Loss 0.683086 |  Val Loss 0.700584
Epoch 17: Train Loss 0.686362 |  Val Los

[I 2025-01-04 15:22:17,635] Trial 0 finished with value: 0.2412940100777387 and parameters: {'n_trees': 16, 'n_selected': 16, 'n_layers': 1, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.2412940100777387.


Epoch 279: Train Loss 0.542687 |  Val Loss 0.604932
Epoch 280: Train Loss 0.546956 |  Val Loss 0.605129
Epoch 281: Train Loss 0.546791 |  Val Loss 0.605453
Epoch 282: Train Loss 0.544015 |  Val Loss 0.605996
Epoch 283: Train Loss 0.547916 |  Val Loss 0.606751
Epoch 284: Train Loss 0.534550 |  Val Loss 0.607048
Epoch 285: Train Loss 0.536540 |  Val Loss 0.605931
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.743236 |  Val Loss 0.703077
Epoch 1: Train Loss 0.674330 |  Val Loss 0.665549
Epoch 2: Train Loss 0.627165 |  Val Loss 0.643837
Epoch 3: Train Loss 0.609267 |  Val Loss 0.633130
Epoch 4: Train Loss 0.583583 |  Val Loss 0.626664
Epoch 5: Train Loss 0.584485 |  Val Loss 0.619731
Epoch 6: Train Loss 0.580738 |  Val Loss 0.614020
Epoch 7: Train Loss 0.551243 |  Val Loss 0.603537
Epoch 8: Train Loss 0.546854 |  Val Loss 0.594604
Epoch 9: Train Loss 0.535755 |  Val Loss 0.589301
Epoch 10: Train Loss 0.5

[I 2025-01-04 15:22:19,654] Trial 1 finished with value: 0.7156282958244218 and parameters: {'n_trees': 16, 'n_selected': 24, 'n_layers': 5, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.7156282958244218.


...

Epoch 35: Train Loss 0.170228 |  Val Loss 0.439219
Epoch 36: Train Loss 0.142814 |  Val Loss 0.439868
Epoch 37: Train Loss 0.149444 |  Val Loss 0.454254
Epoch 38: Train Loss 0.138401 |  Val Loss 0.488629
Epoch 39: Train Loss 0.137021 |  Val Loss 0.513564
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.692464 |  Val Loss 0.676845
Epoch 1: Train Loss 0.671863 |  Val Loss 0.665567
Epoch 2: Train Loss 0.655128 |  Val Loss 0.659429
Epoch 3: Train Loss 0.649019 |  Val Loss 0.654250
Epoch 4: Train Loss 0.638621 |  Val Loss 0.650182
Epoch 5: Train Loss 0.638340 |  Val Loss 0.647013
Epoch 6: Train Loss 0.634093 |  Val Loss 0.642758
Epoch 7: Train Loss 0.624683 |  Val Loss 0.638336
Epoch 8: Train Loss 0.619508 |  Val Loss 0.633941
Epoch 9: Train Loss 0.615016 |  Val Loss 0.629719
Epoch 10: Train Loss 0.601317 |  Val Loss 0.625302
Epoch 11: Train Loss 0.591658 |  Val Loss 0.621039
Epoch 12: Train Loss 0.598791 |

[I 2025-01-04 15:23:07,163] Trial 13 finished with value: 0.6424766100528891 and parameters: {'n_trees': 24, 'n_selected': 16, 'n_layers': 3, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.7156282958244218.


Epoch 79: Train Loss 0.173531 |  Val Loss 0.402859
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.692649 |  Val Loss 0.689945
Epoch 1: Train Loss 0.687853 |  Val Loss 0.687347
Epoch 2: Train Loss 0.683008 |  Val Loss 0.685713
Epoch 3: Train Loss 0.682081 |  Val Loss 0.684236
Epoch 4: Train Loss 0.679261 |  Val Loss 0.682553
Epoch 5: Train Loss 0.676786 |  Val Loss 0.681057
Epoch 6: Train Loss 0.675587 |  Val Loss 0.679476
Epoch 7: Train Loss 0.671155 |  Val Loss 0.677666
Epoch 8: Train Loss 0.669406 |  Val Loss 0.676020
Epoch 9: Train Loss 0.668456 |  Val Loss 0.674264
Epoch 10: Train Loss 0.662069 |  Val Loss 0.672233
Epoch 11: Train Loss 0.659042 |  Val Loss 0.670256
Epoch 12: Train Loss 0.659605 |  Val Loss 0.668236
Epoch 13: Train Loss 0.656989 |  Val Loss 0.666413
Epoch 14: Train Loss 0.651438 |  Val Loss 0.664592
Epoch 15: Train Loss 0.650822 |  Val Loss 0.662087
Epoch 16: Train Loss 0.647460 |

[I 2025-01-04 15:23:11,017] Trial 14 finished with value: 0.40364564761568605 and parameters: {'n_trees': 56, 'n_selected': 4, 'n_layers': 5, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.7156282958244218.


Epoch 110: Train Loss 0.226970 |  Val Loss 0.450786
Epoch 111: Train Loss 0.221296 |  Val Loss 0.453544
Epoch 112: Train Loss 0.213988 |  Val Loss 0.433682
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 16, 'n_selected': 24, 'n_layers': 5, 'mask_type': 'sparsemax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.743236 |  Val Loss 0.703077
Epoch 1: Train Loss 0.674330 |  Val Loss 0.665549
Epoch 2: Train Loss 0.627165 |  Val Loss 0.643837
Epoch 3: Train Loss 0.609267 |  Val Loss 0.633130
Epoch 4: Train Loss 0.583583 |  Val Loss 0.626664
Epoch 5: Train Loss 0.584485 |  Val Loss 0.619731
Epoch 6: Train Loss 0.580738 |  Val Loss 0.614020
Epoch 7: Train Loss 0.551243 |  Val Loss 0.603537
Epoch 8: Train Loss 0.546854 |  Val Loss 0.594604
Epoch 9: Train Loss 0.535755 |  Val Loss 0.589301
Epoch 10: Train Loss 0.506420 |  Val Loss 0.583109
Epoch 11: Train Loss 0.499074 |  Val Loss 0.573068
Epoch 12: Train Loss 0.506186 |  

In [24]:
start_time = time.time()
IX_all_pcc95_results_df = train_with_best_hyperparameters(datasets, 'IX_all_pcc95')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

Currently training NCART model with dataset key: IX_all_pcc95


[I 2025-01-04 15:23:27,219] A new study created in memory with name: no-name-1183cbe0-4fcd-4bf0-b947-bc9bb9a57c9c


On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.844798 |  Val Loss 0.784966
Epoch 1: Train Loss 0.774845 |  Val Loss 0.726062
Epoch 2: Train Loss 0.710036 |  Val Loss 0.683840
Epoch 3: Train Loss 0.666190 |  Val Loss 0.660394
Epoch 4: Train Loss 0.636558 |  Val Loss 0.650420
Epoch 5: Train Loss 0.637355 |  Val Loss 0.647924
Epoch 6: Train Loss 0.633349 |  Val Loss 0.648295
Epoch 7: Train Loss 0.621157 |  Val Loss 0.646349
Epoch 8: Train Loss 0.626302 |  Val Loss 0.641823
Epoch 9: Train Loss 0.616088 |  Val Loss 0.633038
Epoch 10: Train Loss 0.598849 |  Val Loss 0.623554
Epoch 11: Train Loss 0.582062 |  Val Loss 0.615234
Epoch 12: Train Loss 0.575205 |  Val Loss 0.606838
Epoch 13: Train Loss 0.568830 |  Val Loss 0.601762
Epoch 14: Train Loss 0.544479 |  Val Loss 0.597561
Epoch 15: Train Loss 0.542500 |  Val Loss 0.593799
Epoch 16: Train Loss 0.527098 |  Val Loss 0.587107
Epoch 17: Train Loss 0.522422 |  Val Loss 0.577940
Epoch 18: Train Loss 0.505879 |  Val Loss 0.568350


[I 2025-01-04 15:23:28,735] Trial 0 finished with value: 0.3978753504812128 and parameters: {'n_trees': 88, 'n_selected': 24, 'n_layers': 4, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.3978753504812128.


Epoch 46: Train Loss 0.173551 |  Val Loss 0.527261
Epoch 47: Train Loss 0.177296 |  Val Loss 0.556603
Epoch 48: Train Loss 0.170100 |  Val Loss 0.577555
Epoch 49: Train Loss 0.161237 |  Val Loss 0.562084
Epoch 50: Train Loss 0.172607 |  Val Loss 0.571811
Epoch 51: Train Loss 0.157116 |  Val Loss 0.609287
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.694277 |  Val Loss 0.689931
Epoch 1: Train Loss 0.681976 |  Val Loss 0.681570
Epoch 2: Train Loss 0.667958 |  Val Loss 0.673310
Epoch 3: Train Loss 0.663980 |  Val Loss 0.668393
Epoch 4: Train Loss 0.656398 |  Val Loss 0.662982
Epoch 5: Train Loss 0.651360 |  Val Loss 0.658035
Epoch 6: Train Loss 0.647084 |  Val Loss 0.652897
Epoch 7: Train Loss 0.635102 |  Val Loss 0.649311
Epoch 8: Train Loss 0.639628 |  Val Loss 0.646623
Epoch 9: Train Loss 0.626222 |  Val Loss 0.643809
Epoch 10: Train Loss 0.618416 |  Val Loss 0.640652
Epoch 11: Train Loss 0.612334 |

...

Epoch 134: Train Loss 0.240389 |  Val Loss 0.478375
Epoch 135: Train Loss 0.232915 |  Val Loss 0.483006
Epoch 136: Train Loss 0.216721 |  Val Loss 0.479884
Epoch 137: Train Loss 0.231938 |  Val Loss 0.478007
Epoch 138: Train Loss 0.216557 |  Val Loss 0.479865
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.711898 |  Val Loss 0.696967
Epoch 1: Train Loss 0.696890 |  Val Loss 0.685142
Epoch 2: Train Loss 0.682145 |  Val Loss 0.675514
Epoch 3: Train Loss 0.670455 |  Val Loss 0.668020
Epoch 4: Train Loss 0.653043 |  Val Loss 0.662078
Epoch 5: Train Loss 0.653225 |  Val Loss 0.657147
Epoch 6: Train Loss 0.645561 |  Val Loss 0.651607
Epoch 7: Train Loss 0.632530 |  Val Loss 0.647722
Epoch 8: Train Loss 0.632619 |  Val Loss 0.644560
Epoch 9: Train Loss 0.618592 |  Val Loss 0.641013
Epoch 10: Train Loss 0.608098 |  Val Loss 0.637594
Epoch 11: Train Loss 0.600300 |  Val Loss 0.632280
Epoch 12: Train Loss 0.595

[I 2025-01-04 15:24:10,715] Trial 13 finished with value: 0.5444110367045858 and parameters: {'n_trees': 128, 'n_selected': 8, 'n_layers': 6, 'mask_type': 'sparsemax'}. Best is trial 4 with value: 0.6127062015541775.


On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.743729 |  Val Loss 0.746279
Epoch 1: Train Loss 0.722457 |  Val Loss 0.730638
Epoch 2: Train Loss 0.702337 |  Val Loss 0.716933
Epoch 3: Train Loss 0.698473 |  Val Loss 0.705911
Epoch 4: Train Loss 0.689567 |  Val Loss 0.696667
Epoch 5: Train Loss 0.676942 |  Val Loss 0.689380
Epoch 6: Train Loss 0.671827 |  Val Loss 0.683694
Epoch 7: Train Loss 0.663960 |  Val Loss 0.679024
Epoch 8: Train Loss 0.662121 |  Val Loss 0.675375
Epoch 9: Train Loss 0.656408 |  Val Loss 0.672560
Epoch 10: Train Loss 0.652822 |  Val Loss 0.670350
Epoch 11: Train Loss 0.649678 |  Val Loss 0.668412
Epoch 12: Train Loss 0.650316 |  Val Loss 0.665482
Epoch 13: Train Loss 0.648490 |  Val Loss 0.663230
Epoch 14: Train Loss 0.640935 |  Val Loss 0.661027
Epoch 15: Train Loss 0.643243 |  Val Loss 0.659217
Epoch 16: Train Loss 0.636111 |  Val Loss 0.657301
Epoch 17: Train Loss 0.638637 |  Val Loss 0.654994
Epoch 18: Train Loss 0.629731 |  Val Loss 0.652901


[I 2025-01-04 15:24:14,814] Trial 14 finished with value: 0.5879703468219306 and parameters: {'n_trees': 48, 'n_selected': 12, 'n_layers': 2, 'mask_type': 'entmax'}. Best is trial 4 with value: 0.6127062015541775.


Epoch 145: Train Loss 0.207422 |  Val Loss 0.505384
Epoch 146: Train Loss 0.207607 |  Val Loss 0.507026
Epoch 147: Train Loss 0.207266 |  Val Loss 0.515624
Epoch 148: Train Loss 0.200352 |  Val Loss 0.521895
Epoch 149: Train Loss 0.194690 |  Val Loss 0.525915
Epoch 150: Train Loss 0.184038 |  Val Loss 0.521546
Epoch 151: Train Loss 0.182418 |  Val Loss 0.519532
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 72, 'n_selected': 8, 'n_layers': 3, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.689325 |  Val Loss 0.688847
Epoch 1: Train Loss 0.684358 |  Val Loss 0.685522
Epoch 2: Train Loss 0.678918 |  Val Loss 0.682765
Epoch 3: Train Loss 0.678197 |  Val Loss 0.680799
Epoch 4: Train Loss 0.673518 |  Val Loss 0.678522
Epoch 5: Train Loss 0.673339 |  Val Loss 0.677073
Epoch 6: Train Loss 0.670775 |  Val Loss 0.675522
Epoch 7: Train Loss 0.665950 |  Val Loss 0.673605
Epoch 8: Train Loss 0.667768 | 

In [25]:
start_time = time.time()
IX_all_pcc75_results_df = train_with_best_hyperparameters(datasets, 'IX_all_pcc75')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

[I 2025-01-04 15:24:44,222] A new study created in memory with name: no-name-3fef27e2-9333-4aed-8d7a-dc8c2c77d75c


Currently training NCART model with dataset key: IX_all_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.772730 |  Val Loss 0.759533
Epoch 1: Train Loss 0.757255 |  Val Loss 0.746256
Epoch 2: Train Loss 0.743793 |  Val Loss 0.731858
Epoch 3: Train Loss 0.722672 |  Val Loss 0.720796
Epoch 4: Train Loss 0.699105 |  Val Loss 0.711122
Epoch 5: Train Loss 0.698245 |  Val Loss 0.701860
Epoch 6: Train Loss 0.686494 |  Val Loss 0.692932
Epoch 7: Train Loss 0.671666 |  Val Loss 0.684492
Epoch 8: Train Loss 0.669229 |  Val Loss 0.676506
Epoch 9: Train Loss 0.655402 |  Val Loss 0.669243
Epoch 10: Train Loss 0.648969 |  Val Loss 0.662811
Epoch 11: Train Loss 0.647311 |  Val Loss 0.656627
Epoch 12: Train Loss 0.638366 |  Val Loss 0.651153
Epoch 13: Train Loss 0.632043 |  Val Loss 0.646644
Epoch 14: Train Loss 0.619440 |  Val Loss 0.642563
Epoch 15: Train Loss 0.625286 |  Val Loss 0.639209
Epoch 16: Train Loss 0.611129 |  Val Loss 0.635499
Epoch 17: Train Loss 0.608811 |  Val Los

[I 2025-01-04 15:24:46,949] Trial 0 finished with value: 0.5048751223685388 and parameters: {'n_trees': 8, 'n_selected': 8, 'n_layers': 8, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.5048751223685388.


Epoch 60: Train Loss 0.288395 |  Val Loss 0.539881
Epoch 61: Train Loss 0.296315 |  Val Loss 0.543187
Epoch 62: Train Loss 0.287026 |  Val Loss 0.545082
Epoch 63: Train Loss 0.280265 |  Val Loss 0.539911
Epoch 64: Train Loss 0.263799 |  Val Loss 0.546474
Epoch 65: Train Loss 0.260855 |  Val Loss 0.553413
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.704636 |  Val Loss 0.690921
Epoch 1: Train Loss 0.683993 |  Val Loss 0.677529
Epoch 2: Train Loss 0.666012 |  Val Loss 0.669114
Epoch 3: Train Loss 0.662749 |  Val Loss 0.661635
Epoch 4: Train Loss 0.653604 |  Val Loss 0.654594
Epoch 5: Train Loss 0.648234 |  Val Loss 0.650485
Epoch 6: Train Loss 0.645715 |  Val Loss 0.646104
Epoch 7: Train Loss 0.629332 |  Val Loss 0.640458
Epoch 8: Train Loss 0.629733 |  Val Loss 0.635950
Epoch 9: Train Loss 0.614597 |  Val Loss 0.631104
Epoch 10: Train Loss 0.609137 |  Val Loss 0.626782
Epoch 11: Train Loss 0.601325 |

[I 2025-01-04 15:24:48,962] Trial 1 finished with value: 0.5637602382548061 and parameters: {'n_trees': 112, 'n_selected': 16, 'n_layers': 5, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.5637602382548061.


...

Epoch 88: Train Loss 0.192664 |  Val Loss 0.523614
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 1.434186 |  Val Loss 1.275374
Epoch 1: Train Loss 1.185386 |  Val Loss 1.057637
Epoch 2: Train Loss 0.960724 |  Val Loss 0.883221
Epoch 3: Train Loss 0.819910 |  Val Loss 0.756972
Epoch 4: Train Loss 0.712461 |  Val Loss 0.683713
Epoch 5: Train Loss 0.642686 |  Val Loss 0.657798
Epoch 6: Train Loss 0.632428 |  Val Loss 0.660895
Epoch 7: Train Loss 0.628664 |  Val Loss 0.674605
Epoch 8: Train Loss 0.654245 |  Val Loss 0.683532
Epoch 9: Train Loss 0.633727 |  Val Loss 0.682706
Epoch 10: Train Loss 0.636319 |  Val Loss 0.674047
Epoch 11: Train Loss 0.626444 |  Val Loss 0.660046
Epoch 12: Train Loss 0.609086 |  Val Loss 0.644973
Epoch 13: Train Loss 0.578302 |  Val Loss 0.633398
Epoch 14: Train Loss 0.557534 |  Val Loss 0.627323
Epoch 15: Train Loss 0.562158 |  Val Loss 0.624826
Epoch 16: Train Loss 0.548464 |

[I 2025-01-04 15:25:21,205] Trial 13 finished with value: 0.5316441225967666 and parameters: {'n_trees': 32, 'n_selected': 32, 'n_layers': 4, 'mask_type': 'sparsemax'}. Best is trial 7 with value: 0.6666497001383711.


Epoch 56: Train Loss 0.242640 |  Val Loss 0.538304
Epoch 57: Train Loss 0.245950 |  Val Loss 0.550030
Epoch 58: Train Loss 0.234131 |  Val Loss 0.549762
Epoch 59: Train Loss 0.231349 |  Val Loss 0.548087
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.724905 |  Val Loss 0.714913
Epoch 1: Train Loss 0.707421 |  Val Loss 0.704496
Epoch 2: Train Loss 0.692798 |  Val Loss 0.696268
Epoch 3: Train Loss 0.689587 |  Val Loss 0.688846
Epoch 4: Train Loss 0.685695 |  Val Loss 0.681955
Epoch 5: Train Loss 0.671030 |  Val Loss 0.675267
Epoch 6: Train Loss 0.667216 |  Val Loss 0.668925
Epoch 7: Train Loss 0.657363 |  Val Loss 0.663343
Epoch 8: Train Loss 0.654672 |  Val Loss 0.658473
Epoch 9: Train Loss 0.647073 |  Val Loss 0.654735
Epoch 10: Train Loss 0.639424 |  Val Loss 0.651020
Epoch 11: Train Loss 0.633328 |  Val Loss 0.647778
Epoch 12: Train Loss 0.631211 |  Val Loss 0.645080
Epoch 13: Train Loss 0.623391 |

[I 2025-01-04 15:25:23,575] Trial 14 finished with value: 0.4930139694215446 and parameters: {'n_trees': 64, 'n_selected': 8, 'n_layers': 6, 'mask_type': 'sparsemax'}. Best is trial 7 with value: 0.6666497001383711.


Epoch 60: Train Loss 0.239734 |  Val Loss 0.540066
Epoch 61: Train Loss 0.224808 |  Val Loss 0.538810
Epoch 62: Train Loss 0.232238 |  Val Loss 0.531145
Epoch 63: Train Loss 0.212898 |  Val Loss 0.543955
Epoch 64: Train Loss 0.205009 |  Val Loss 0.550458
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 56, 'n_selected': 4, 'n_layers': 7, 'mask_type': 'sparsemax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.700168 |  Val Loss 0.699535
Epoch 1: Train Loss 0.693749 |  Val Loss 0.696774
Epoch 2: Train Loss 0.688871 |  Val Loss 0.694069
Epoch 3: Train Loss 0.689468 |  Val Loss 0.691783
Epoch 4: Train Loss 0.689331 |  Val Loss 0.689612
Epoch 5: Train Loss 0.683017 |  Val Loss 0.687158
Epoch 6: Train Loss 0.681764 |  Val Loss 0.684867
Epoch 7: Train Loss 0.676642 |  Val Loss 0.682623
Epoch 8: Train Loss 0.675908 |  Val Loss 0.680176
Epoch 9: Train Loss 0.674087 |  Val Loss 0.678047
Epoch 10: Train Loss 0.669904 |  Val 

In [26]:
start_time = time.time()
IX_500_nopcc_results_df = train_with_best_hyperparameters(datasets, 'IX_500_nopcc')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

[I 2025-01-04 15:26:06,850] A new study created in memory with name: no-name-fda2a6d9-c4e8-413d-a7b7-5ac3bcf62c3b


Currently training NCART model with dataset key: IX_500_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.788959 |  Val Loss 0.742462
Epoch 1: Train Loss 0.762895 |  Val Loss 0.719281
Epoch 2: Train Loss 0.740336 |  Val Loss 0.699724
Epoch 3: Train Loss 0.721454 |  Val Loss 0.683945
Epoch 4: Train Loss 0.706342 |  Val Loss 0.671874
Epoch 5: Train Loss 0.694943 |  Val Loss 0.663342
Epoch 6: Train Loss 0.687055 |  Val Loss 0.658007
Epoch 7: Train Loss 0.682274 |  Val Loss 0.655332
Epoch 8: Train Loss 0.680053 |  Val Loss 0.654658
Epoch 9: Train Loss 0.679675 |  Val Loss 0.655221
Epoch 10: Train Loss 0.680351 |  Val Loss 0.656294
Epoch 11: Train Loss 0.681341 |  Val Loss 0.657257
Epoch 12: Train Loss 0.682041 |  Val Loss 0.657708
Epoch 13: Train Loss 0.682057 |  Val Loss 0.657451
Epoch 14: Train Loss 0.681207 |  Val Loss 0.656485
Epoch 15: Train Loss 0.679497 |  Val Loss 0.654883
Epoch 16: Train Loss 0.677051 |  Val Loss 0.652830
Epoch 17: Train Loss 0.674076 |  Val Los

[I 2025-01-04 15:26:17,974] Trial 0 finished with value: 0.40522044923655387 and parameters: {'n_trees': 16, 'n_selected': 20, 'n_layers': 1, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.40522044923655387.


Epoch 776: Train Loss 0.336680 |  Val Loss 0.543041
Epoch 777: Train Loss 0.336257 |  Val Loss 0.543020
Epoch 778: Train Loss 0.335835 |  Val Loss 0.543003
Epoch 779: Train Loss 0.335416 |  Val Loss 0.542983
Epoch 780: Train Loss 0.334996 |  Val Loss 0.542973
Epoch 781: Train Loss 0.334573 |  Val Loss 0.542998
Epoch 782: Train Loss 0.334147 |  Val Loss 0.543022
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.795199 |  Val Loss 0.767431
Epoch 1: Train Loss 0.751305 |  Val Loss 0.741393
Epoch 2: Train Loss 0.720169 |  Val Loss 0.720908
Epoch 3: Train Loss 0.692212 |  Val Loss 0.702740
Epoch 4: Train Loss 0.668162 |  Val Loss 0.686271
Epoch 5: Train Loss 0.645015 |  Val Loss 0.671141
Epoch 6: Train Loss 0.623403 |  Val Loss 0.658906
Epoch 7: Train Loss 0.602890 |  Val Loss 0.649405
Epoch 8: Train Loss 0.583725 |  Val Loss 0.641059
Epoch 9: Train Loss 0.566301 |  Val Loss 0.631638
Epoch 10: Train Loss 0.5

[I 2025-01-04 15:26:19,483] Trial 1 finished with value: 0.5857689433539132 and parameters: {'n_trees': 48, 'n_selected': 16, 'n_layers': 8, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.5857689433539132.


...

Epoch 38: Train Loss 0.300806 |  Val Loss 0.532665
Epoch 39: Train Loss 0.291164 |  Val Loss 0.534035
Epoch 40: Train Loss 0.282627 |  Val Loss 0.539367
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.920327 |  Val Loss 0.874944
Epoch 1: Train Loss 0.880302 |  Val Loss 0.844168
Epoch 2: Train Loss 0.845129 |  Val Loss 0.816069
Epoch 3: Train Loss 0.813978 |  Val Loss 0.791448
Epoch 4: Train Loss 0.786685 |  Val Loss 0.767742
Epoch 5: Train Loss 0.761286 |  Val Loss 0.747043
Epoch 6: Train Loss 0.738201 |  Val Loss 0.728754
Epoch 7: Train Loss 0.716839 |  Val Loss 0.711403
Epoch 8: Train Loss 0.696848 |  Val Loss 0.696359
Epoch 9: Train Loss 0.678348 |  Val Loss 0.682802
Epoch 10: Train Loss 0.661460 |  Val Loss 0.670862
Epoch 11: Train Loss 0.645813 |  Val Loss 0.659999
Epoch 12: Train Loss 0.631466 |  Val Loss 0.650467
Epoch 13: Train Loss 0.618282 |  Val Loss 0.642701
Epoch 14: Train Loss 0.606032 |

[I 2025-01-04 15:26:42,900] Trial 13 finished with value: 0.5401080324108039 and parameters: {'n_trees': 40, 'n_selected': 16, 'n_layers': 5, 'mask_type': 'sparsemax'}. Best is trial 10 with value: 0.6420578831241024.


Epoch 53: Train Loss 0.248875 |  Val Loss 0.479071
Epoch 54: Train Loss 0.241083 |  Val Loss 0.480498
Epoch 55: Train Loss 0.232631 |  Val Loss 0.485230
Epoch 56: Train Loss 0.225314 |  Val Loss 0.485048
Epoch 57: Train Loss 0.216809 |  Val Loss 0.483322
Epoch 58: Train Loss 0.211118 |  Val Loss 0.483763
Epoch 59: Train Loss 0.202959 |  Val Loss 0.490571
Epoch 60: Train Loss 0.196182 |  Val Loss 0.490172
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.706406 |  Val Loss 0.700705
Epoch 1: Train Loss 0.694669 |  Val Loss 0.694174
Epoch 2: Train Loss 0.685031 |  Val Loss 0.687545
Epoch 3: Train Loss 0.676597 |  Val Loss 0.681988
Epoch 4: Train Loss 0.669327 |  Val Loss 0.677808
Epoch 5: Train Loss 0.662393 |  Val Loss 0.672900
Epoch 6: Train Loss 0.655604 |  Val Loss 0.668477
Epoch 7: Train Loss 0.648804 |  Val Loss 0.664316
Epoch 8: Train Loss 0.642115 |  Val Loss 0.660540
Epoch 9: Train Loss 0.635595 |

[I 2025-01-04 15:26:45,375] Trial 14 finished with value: 0.6043672230190352 and parameters: {'n_trees': 80, 'n_selected': 8, 'n_layers': 7, 'mask_type': 'entmax'}. Best is trial 10 with value: 0.6420578831241024.


Epoch 78: Train Loss 0.245202 |  Val Loss 0.552925
Epoch 79: Train Loss 0.241625 |  Val Loss 0.554259
Epoch 80: Train Loss 0.237882 |  Val Loss 0.559428
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 88, 'n_selected': 12, 'n_layers': 6, 'mask_type': 'sparsemax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.726652 |  Val Loss 0.716243
Epoch 1: Train Loss 0.707942 |  Val Loss 0.704601
Epoch 2: Train Loss 0.694043 |  Val Loss 0.693959
Epoch 3: Train Loss 0.680947 |  Val Loss 0.684547
Epoch 4: Train Loss 0.669583 |  Val Loss 0.676618
Epoch 5: Train Loss 0.658787 |  Val Loss 0.667440
Epoch 6: Train Loss 0.648367 |  Val Loss 0.661102
Epoch 7: Train Loss 0.638806 |  Val Loss 0.654838
Epoch 8: Train Loss 0.630306 |  Val Loss 0.649572
Epoch 9: Train Loss 0.621979 |  Val Loss 0.644710
Epoch 10: Train Loss 0.613859 |  Val Loss 0.640936
Epoch 11: Train Loss 0.606070 |  Val Loss 0.637248
Epoch 12: Train Loss 0.598682 |  Val

In [27]:
start_time = time.time()
IX_500_pcc95_results_df = train_with_best_hyperparameters(datasets, 'IX_500_pcc95')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

[I 2025-01-04 15:27:04,714] A new study created in memory with name: no-name-67664e9f-fbb1-4a18-85bf-e22a3af30757


Currently training NCART model with dataset key: IX_500_pcc95
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 1.160426 |  Val Loss 1.112264
Epoch 1: Train Loss 1.062092 |  Val Loss 1.026788
Epoch 2: Train Loss 0.973470 |  Val Loss 0.951891
Epoch 3: Train Loss 0.893631 |  Val Loss 0.888186
Epoch 4: Train Loss 0.821783 |  Val Loss 0.835376
Epoch 5: Train Loss 0.758609 |  Val Loss 0.788209
Epoch 6: Train Loss 0.703141 |  Val Loss 0.748545
Epoch 7: Train Loss 0.655688 |  Val Loss 0.713937
Epoch 8: Train Loss 0.617056 |  Val Loss 0.687829
Epoch 9: Train Loss 0.586009 |  Val Loss 0.666853
Epoch 10: Train Loss 0.561474 |  Val Loss 0.650985
Epoch 11: Train Loss 0.542883 |  Val Loss 0.637965
Epoch 12: Train Loss 0.529707 |  Val Loss 0.630249
Epoch 13: Train Loss 0.520220 |  Val Loss 0.623463
Epoch 14: Train Loss 0.513600 |  Val Loss 0.619590
Epoch 15: Train Loss 0.508564 |  Val Loss 0.618261
Epoch 16: Train Loss 0.504193 |  Val Loss 0.616675
Epoch 17: Train Loss 0.499512 |  Val Los

[I 2025-01-04 15:27:05,918] Trial 0 finished with value: 0.6201240372124044 and parameters: {'n_trees': 16, 'n_selected': 24, 'n_layers': 5, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.6201240372124044.


Epoch 45: Train Loss 0.237057 |  Val Loss 0.487637
Epoch 46: Train Loss 0.229673 |  Val Loss 0.492326
Epoch 47: Train Loss 0.222022 |  Val Loss 0.497836
Epoch 48: Train Loss 0.214902 |  Val Loss 0.500670
Epoch 49: Train Loss 0.207536 |  Val Loss 0.504207
Epoch 50: Train Loss 0.200570 |  Val Loss 0.511558
Epoch 51: Train Loss 0.193762 |  Val Loss 0.514648
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.715812 |  Val Loss 0.705557
Epoch 1: Train Loss 0.707852 |  Val Loss 0.699754
Epoch 2: Train Loss 0.701948 |  Val Loss 0.695912
Epoch 3: Train Loss 0.698015 |  Val Loss 0.693780
Epoch 4: Train Loss 0.695818 |  Val Loss 0.692912
Epoch 5: Train Loss 0.694920 |  Val Loss 0.692719
Epoch 6: Train Loss 0.694736 |  Val Loss 0.692628
Epoch 7: Train Loss 0.694687 |  Val Loss 0.692229
Epoch 8: Train Loss 0.694356 |  Val Loss 0.691355
Epoch 9: Train Loss 0.693567 |  Val Loss 0.690029
Epoch 10: Train Loss 0.692336 |

[I 2025-01-04 15:27:17,581] Trial 1 finished with value: 0.601929265428846 and parameters: {'n_trees': 112, 'n_selected': 20, 'n_layers': 1, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.6201240372124044.


...

Epoch 61: Train Loss 0.300369 |  Val Loss 0.540798
Epoch 62: Train Loss 0.295888 |  Val Loss 0.546822
Epoch 63: Train Loss 0.290409 |  Val Loss 0.549268
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.691053 |  Val Loss 0.682833
Epoch 1: Train Loss 0.668510 |  Val Loss 0.679201
Epoch 2: Train Loss 0.658113 |  Val Loss 0.673230
Epoch 3: Train Loss 0.649155 |  Val Loss 0.665201
Epoch 4: Train Loss 0.639309 |  Val Loss 0.656527
Epoch 5: Train Loss 0.628654 |  Val Loss 0.648501
Epoch 6: Train Loss 0.617815 |  Val Loss 0.640606
Epoch 7: Train Loss 0.608824 |  Val Loss 0.632610
Epoch 8: Train Loss 0.599481 |  Val Loss 0.626098
Epoch 9: Train Loss 0.589536 |  Val Loss 0.617508
Epoch 10: Train Loss 0.578275 |  Val Loss 0.608953
Epoch 11: Train Loss 0.566595 |  Val Loss 0.599317
Epoch 12: Train Loss 0.554452 |  Val Loss 0.589224
Epoch 13: Train Loss 0.541185 |  Val Loss 0.581041
Epoch 14: Train Loss 0.526997 |

[I 2025-01-04 15:27:46,984] Trial 13 finished with value: 0.6821865008193587 and parameters: {'n_trees': 128, 'n_selected': 28, 'n_layers': 6, 'mask_type': 'entmax'}. Best is trial 11 with value: 0.6821865008193587.


Epoch 38: Train Loss 0.154288 |  Val Loss 0.479159
Epoch 39: Train Loss 0.147866 |  Val Loss 0.492186
Epoch 40: Train Loss 0.138043 |  Val Loss 0.495761
Epoch 41: Train Loss 0.131629 |  Val Loss 0.503574
Epoch 42: Train Loss 0.122404 |  Val Loss 0.502587
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.721689 |  Val Loss 0.705533
Epoch 1: Train Loss 0.692690 |  Val Loss 0.693551
Epoch 2: Train Loss 0.676292 |  Val Loss 0.687733
Epoch 3: Train Loss 0.665705 |  Val Loss 0.679445
Epoch 4: Train Loss 0.655966 |  Val Loss 0.671113
Epoch 5: Train Loss 0.645001 |  Val Loss 0.663601
Epoch 6: Train Loss 0.634303 |  Val Loss 0.655819
Epoch 7: Train Loss 0.623661 |  Val Loss 0.649025
Epoch 8: Train Loss 0.613892 |  Val Loss 0.641675
Epoch 9: Train Loss 0.604651 |  Val Loss 0.634707
Epoch 10: Train Loss 0.594792 |  Val Loss 0.627096
Epoch 11: Train Loss 0.584176 |  Val Loss 0.619666
Epoch 12: Train Loss 0.573425 |

[I 2025-01-04 15:27:48,175] Trial 14 finished with value: 0.6211190222681673 and parameters: {'n_trees': 80, 'n_selected': 28, 'n_layers': 6, 'mask_type': 'entmax'}. Best is trial 11 with value: 0.6821865008193587.


Epoch 35: Train Loss 0.221888 |  Val Loss 0.514642
Epoch 36: Train Loss 0.207841 |  Val Loss 0.525634
Epoch 37: Train Loss 0.194501 |  Val Loss 0.536542
Epoch 38: Train Loss 0.184465 |  Val Loss 0.552331
Epoch 39: Train Loss 0.173415 |  Val Loss 0.545111
Epoch 40: Train Loss 0.160053 |  Val Loss 0.560712
Epoch 41: Train Loss 0.150246 |  Val Loss 0.575959
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 128, 'n_selected': 28, 'n_layers': 6, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.691053 |  Val Loss 0.682833
Epoch 1: Train Loss 0.668510 |  Val Loss 0.679201
Epoch 2: Train Loss 0.658113 |  Val Loss 0.673230
Epoch 3: Train Loss 0.649155 |  Val Loss 0.665201
Epoch 4: Train Loss 0.639309 |  Val Loss 0.656527
Epoch 5: Train Loss 0.628654 |  Val Loss 0.648501
Epoch 6: Train Loss 0.617815 |  Val Loss 0.640606
Epoch 7: Train Loss 0.608824 |  Val Loss 0.632610
Epoch 8: Train Loss 0.599481 |  Val 

In [28]:
start_time = time.time()
IX_500_pcc75_results_df = train_with_best_hyperparameters(datasets, 'IX_500_pcc75')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

[I 2025-01-04 15:28:03,377] A new study created in memory with name: no-name-7597e053-70b1-49af-9ff5-be7c09ec1b7d


Currently training NCART model with dataset key: IX_500_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 6.330286 |  Val Loss 6.071746
Epoch 1: Train Loss 6.143750 |  Val Loss 5.904972
Epoch 2: Train Loss 5.959821 |  Val Loss 5.740891
Epoch 3: Train Loss 5.778845 |  Val Loss 5.570391
Epoch 4: Train Loss 5.601192 |  Val Loss 5.402493
Epoch 5: Train Loss 5.426318 |  Val Loss 5.238806
Epoch 6: Train Loss 5.252444 |  Val Loss 5.076884
Epoch 7: Train Loss 5.080982 |  Val Loss 4.912384
Epoch 8: Train Loss 4.911454 |  Val Loss 4.749616
Epoch 9: Train Loss 4.743689 |  Val Loss 4.588893
Epoch 10: Train Loss 4.577672 |  Val Loss 4.428784
Epoch 11: Train Loss 4.412546 |  Val Loss 4.269605
Epoch 12: Train Loss 4.248343 |  Val Loss 4.110669
Epoch 13: Train Loss 4.084271 |  Val Loss 3.950825
Epoch 14: Train Loss 3.920624 |  Val Loss 3.791911
Epoch 15: Train Loss 3.757770 |  Val Loss 3.629480
Epoch 16: Train Loss 3.595658 |  Val Loss 3.471209
Epoch 17: Train Loss 3.433943 |  Val Los

[I 2025-01-04 15:28:05,920] Trial 0 finished with value: 0.626166801516252 and parameters: {'n_trees': 16, 'n_selected': 32, 'n_layers': 5, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.626166801516252.


Epoch 115: Train Loss 0.117850 |  Val Loss 0.419013
Epoch 116: Train Loss 0.114421 |  Val Loss 0.412388
Epoch 117: Train Loss 0.110754 |  Val Loss 0.408104
Epoch 118: Train Loss 0.108721 |  Val Loss 0.414446
Epoch 119: Train Loss 0.108209 |  Val Loss 0.415704
Epoch 120: Train Loss 0.105311 |  Val Loss 0.417475
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.722929 |  Val Loss 0.688046
Epoch 1: Train Loss 0.674779 |  Val Loss 0.664789
Epoch 2: Train Loss 0.646934 |  Val Loss 0.655472
Epoch 3: Train Loss 0.633786 |  Val Loss 0.648697
Epoch 4: Train Loss 0.624396 |  Val Loss 0.637852
Epoch 5: Train Loss 0.611816 |  Val Loss 0.622584
Epoch 6: Train Loss 0.596267 |  Val Loss 0.608384
Epoch 7: Train Loss 0.580468 |  Val Loss 0.594287
Epoch 8: Train Loss 0.565910 |  Val Loss 0.585032
Epoch 9: Train Loss 0.552708 |  Val Loss 0.578364
Epoch 10: Train Loss 0.541544 |  Val Loss 0.573029
Epoch 11: Train Loss 0.53

[I 2025-01-04 15:28:07,192] Trial 1 finished with value: 0.625 and parameters: {'n_trees': 72, 'n_selected': 32, 'n_layers': 7, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.626166801516252.


...

Epoch 189: Train Loss 0.172648 |  Val Loss 0.430227
Epoch 190: Train Loss 0.170847 |  Val Loss 0.433335
Epoch 191: Train Loss 0.168487 |  Val Loss 0.434285
Epoch 192: Train Loss 0.166460 |  Val Loss 0.429737
Epoch 193: Train Loss 0.164272 |  Val Loss 0.429765
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 1.039186 |  Val Loss 1.003953
Epoch 1: Train Loss 0.987433 |  Val Loss 0.954908
Epoch 2: Train Loss 0.938853 |  Val Loss 0.910833
Epoch 3: Train Loss 0.893444 |  Val Loss 0.868013
Epoch 4: Train Loss 0.851778 |  Val Loss 0.830921
Epoch 5: Train Loss 0.813877 |  Val Loss 0.797714
Epoch 6: Train Loss 0.780338 |  Val Loss 0.768193
Epoch 7: Train Loss 0.750834 |  Val Loss 0.742930
Epoch 8: Train Loss 0.725604 |  Val Loss 0.721877
Epoch 9: Train Loss 0.704392 |  Val Loss 0.705113
Epoch 10: Train Loss 0.687648 |  Val Loss 0.692235
Epoch 11: Train Loss 0.675083 |  Val Loss 0.681962
Epoch 12: Train Loss 0.665

[I 2025-01-04 15:29:10,685] Trial 13 finished with value: 0.626166801516252 and parameters: {'n_trees': 40, 'n_selected': 24, 'n_layers': 2, 'mask_type': 'entmax'}. Best is trial 11 with value: 0.7205766921228921.


Epoch 201: Train Loss 0.164811 |  Val Loss 0.430884
Epoch 202: Train Loss 0.162855 |  Val Loss 0.436013
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.736743 |  Val Loss 0.689396
Epoch 1: Train Loss 0.725624 |  Val Loss 0.680901
Epoch 2: Train Loss 0.715093 |  Val Loss 0.673483
Epoch 3: Train Loss 0.705679 |  Val Loss 0.666859
Epoch 4: Train Loss 0.697344 |  Val Loss 0.660821
Epoch 5: Train Loss 0.689713 |  Val Loss 0.655368
Epoch 6: Train Loss 0.682906 |  Val Loss 0.650729
Epoch 7: Train Loss 0.677023 |  Val Loss 0.647452
Epoch 8: Train Loss 0.672030 |  Val Loss 0.644468
Epoch 9: Train Loss 0.667862 |  Val Loss 0.642214
Epoch 10: Train Loss 0.664199 |  Val Loss 0.640131
Epoch 11: Train Loss 0.660907 |  Val Loss 0.638202
Epoch 12: Train Loss 0.658042 |  Val Loss 0.637031
Epoch 13: Train Loss 0.655488 |  Val Loss 0.635935
Epoch 14: Train Loss 0.653072 |  Val Loss 0.634754
Epoch 15: Train Loss 0.650734

[I 2025-01-04 15:29:13,355] Trial 14 finished with value: 0.5409746322980812 and parameters: {'n_trees': 24, 'n_selected': 12, 'n_layers': 3, 'mask_type': 'entmax'}. Best is trial 11 with value: 0.7205766921228921.


Best hyperparameters: {'n_trees': 40, 'n_selected': 24, 'n_layers': 3, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.795182 |  Val Loss 0.752103
Epoch 1: Train Loss 0.760378 |  Val Loss 0.721973
Epoch 2: Train Loss 0.730998 |  Val Loss 0.698435
Epoch 3: Train Loss 0.706267 |  Val Loss 0.677570
Epoch 4: Train Loss 0.685924 |  Val Loss 0.662629
Epoch 5: Train Loss 0.670591 |  Val Loss 0.651590
Epoch 6: Train Loss 0.659656 |  Val Loss 0.643984
Epoch 7: Train Loss 0.652345 |  Val Loss 0.639261
Epoch 8: Train Loss 0.647674 |  Val Loss 0.636724
Epoch 9: Train Loss 0.644811 |  Val Loss 0.635214
Epoch 10: Train Loss 0.642984 |  Val Loss 0.634121
Epoch 11: Train Loss 0.641255 |  Val Loss 0.632583
Epoch 12: Train Loss 0.638948 |  Val Loss 0.630407
Epoch 13: Train Loss 0.635761 |  Val Loss 0.627236
Epoch 14: Train Loss 0.631612 |  Val Loss 0.622710
Epoch 15: Train Loss 0.626555 |  Val Loss 0.618002
Epoch 16: Train Loss 0.620906 |  Val Loss 0.613240
Epoch 17

In [29]:
start_time = time.time()
IX_400_nopcc_results_df = train_with_best_hyperparameters(datasets, 'IX_400_nopcc')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

[I 2025-01-04 15:29:34,399] A new study created in memory with name: no-name-e120809e-dc88-47ce-9a35-8d325da1c2ad


Currently training NCART model with dataset key: IX_400_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.723534 |  Val Loss 0.722887
Epoch 1: Train Loss 0.717356 |  Val Loss 0.718768
Epoch 2: Train Loss 0.712194 |  Val Loss 0.714123
Epoch 3: Train Loss 0.707648 |  Val Loss 0.709502
Epoch 4: Train Loss 0.703266 |  Val Loss 0.705672
Epoch 5: Train Loss 0.698976 |  Val Loss 0.701752
Epoch 6: Train Loss 0.694729 |  Val Loss 0.697645
Epoch 7: Train Loss 0.690742 |  Val Loss 0.694228
Epoch 8: Train Loss 0.686982 |  Val Loss 0.690909
Epoch 9: Train Loss 0.683541 |  Val Loss 0.687514
Epoch 10: Train Loss 0.680358 |  Val Loss 0.684396
Epoch 11: Train Loss 0.677284 |  Val Loss 0.681373
Epoch 12: Train Loss 0.674366 |  Val Loss 0.678495
Epoch 13: Train Loss 0.671545 |  Val Loss 0.675791
Epoch 14: Train Loss 0.668754 |  Val Loss 0.673249
Epoch 15: Train Loss 0.665996 |  Val Loss 0.670492
Epoch 16: Train Loss 0.663376 |  Val Loss 0.667844
Epoch 17: Train Loss 0.660863 |  Val Los

[I 2025-01-04 15:29:38,032] Trial 0 finished with value: 0.6 and parameters: {'n_trees': 8, 'n_selected': 4, 'n_layers': 3, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.6.


Epoch 225: Train Loss 0.145222 |  Val Loss 0.506000
Epoch 226: Train Loss 0.143801 |  Val Loss 0.503638
Epoch 227: Train Loss 0.142207 |  Val Loss 0.503503
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.744850 |  Val Loss 0.729942
Epoch 1: Train Loss 0.720669 |  Val Loss 0.711626
Epoch 2: Train Loss 0.701208 |  Val Loss 0.697237
Epoch 3: Train Loss 0.685219 |  Val Loss 0.685855
Epoch 4: Train Loss 0.672712 |  Val Loss 0.674670
Epoch 5: Train Loss 0.660549 |  Val Loss 0.663641
Epoch 6: Train Loss 0.649065 |  Val Loss 0.654319
Epoch 7: Train Loss 0.638170 |  Val Loss 0.645614
Epoch 8: Train Loss 0.628038 |  Val Loss 0.638840
Epoch 9: Train Loss 0.618128 |  Val Loss 0.633160
Epoch 10: Train Loss 0.608565 |  Val Loss 0.626124
Epoch 11: Train Loss 0.599148 |  Val Loss 0.619930
Epoch 12: Train Loss 0.589838 |  Val Loss 0.615201
Epoch 13: Train Loss 0.580202 |  Val Loss 0.611099
Epoch 14: Train Loss 0.57037

[I 2025-01-04 15:29:40,316] Trial 1 finished with value: 0.6574382586514258 and parameters: {'n_trees': 112, 'n_selected': 12, 'n_layers': 7, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.6574382586514258.


...

Epoch 61: Train Loss 0.207931 |  Val Loss 0.540841
Epoch 62: Train Loss 0.204066 |  Val Loss 0.539325
Epoch 63: Train Loss 0.198815 |  Val Loss 0.536562
Epoch 64: Train Loss 0.194902 |  Val Loss 0.555132
Epoch 65: Train Loss 0.191052 |  Val Loss 0.545309
Epoch 66: Train Loss 0.186987 |  Val Loss 0.531972
Epoch 67: Train Loss 0.183270 |  Val Loss 0.544782
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.708331 |  Val Loss 0.689214
Epoch 1: Train Loss 0.684940 |  Val Loss 0.674483
Epoch 2: Train Loss 0.667429 |  Val Loss 0.666132
Epoch 3: Train Loss 0.655557 |  Val Loss 0.658219
Epoch 4: Train Loss 0.644053 |  Val Loss 0.649805
Epoch 5: Train Loss 0.633184 |  Val Loss 0.643392
Epoch 6: Train Loss 0.622765 |  Val Loss 0.636903
Epoch 7: Train Loss 0.611692 |  Val Loss 0.631277
Epoch 8: Train Loss 0.600925 |  Val Loss 0.625886
Epoch 9: Train Loss 0.589972 |  Val Loss 0.620389
Epoch 10: Train Loss 0.578856 |

[I 2025-01-04 15:30:13,745] Trial 13 finished with value: 0.6508140266182866 and parameters: {'n_trees': 88, 'n_selected': 12, 'n_layers': 8, 'mask_type': 'sparsemax'}. Best is trial 5 with value: 0.7.


Epoch 54: Train Loss 0.191689 |  Val Loss 0.490975
Epoch 55: Train Loss 0.187091 |  Val Loss 0.489262
Epoch 56: Train Loss 0.182440 |  Val Loss 0.486332
Epoch 57: Train Loss 0.178263 |  Val Loss 0.482354
Epoch 58: Train Loss 0.173789 |  Val Loss 0.484804
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.708402 |  Val Loss 0.702556
Epoch 1: Train Loss 0.703067 |  Val Loss 0.698432
Epoch 2: Train Loss 0.698377 |  Val Loss 0.695008
Epoch 3: Train Loss 0.694580 |  Val Loss 0.692230
Epoch 4: Train Loss 0.690996 |  Val Loss 0.689394
Epoch 5: Train Loss 0.687624 |  Val Loss 0.686484
Epoch 6: Train Loss 0.684291 |  Val Loss 0.683478
Epoch 7: Train Loss 0.680952 |  Val Loss 0.680569
Epoch 8: Train Loss 0.677731 |  Val Loss 0.677698
Epoch 9: Train Loss 0.674558 |  Val Loss 0.675210
Epoch 10: Train Loss 0.671232 |  Val Loss 0.672969
Epoch 11: Train Loss 0.667942 |  Val Loss 0.670858
Epoch 12: Train Loss 0.664613 |

[I 2025-01-04 15:30:16,966] Trial 14 finished with value: 0.5613413993878117 and parameters: {'n_trees': 104, 'n_selected': 4, 'n_layers': 6, 'mask_type': 'sparsemax'}. Best is trial 5 with value: 0.7.


Epoch 118: Train Loss 0.382814 |  Val Loss 0.528296
Epoch 119: Train Loss 0.380464 |  Val Loss 0.531268
Epoch 120: Train Loss 0.378267 |  Val Loss 0.532223
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 120, 'n_selected': 4, 'n_layers': 7, 'mask_type': 'sparsemax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.696596 |  Val Loss 0.693368
Epoch 1: Train Loss 0.691884 |  Val Loss 0.689144
Epoch 2: Train Loss 0.688023 |  Val Loss 0.685923
Epoch 3: Train Loss 0.684359 |  Val Loss 0.683087
Epoch 4: Train Loss 0.681111 |  Val Loss 0.680455
Epoch 5: Train Loss 0.677837 |  Val Loss 0.678002
Epoch 6: Train Loss 0.674488 |  Val Loss 0.675766
Epoch 7: Train Loss 0.671582 |  Val Loss 0.673583
Epoch 8: Train Loss 0.668914 |  Val Loss 0.671605
Epoch 9: Train Loss 0.666243 |  Val Loss 0.669969
Epoch 10: Train Loss 0.663614 |  Val Loss 0.668216
Epoch 11: Train Loss 0.660902 |  Val Loss 0.666538
Epoch 12: Train Loss 0.658116 |  

In [30]:
start_time = time.time()
IX_400_pcc95_results_df = train_with_best_hyperparameters(datasets, 'IX_400_pcc95')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

[I 2025-01-04 15:30:56,405] A new study created in memory with name: no-name-9b6c526a-f0bf-428c-9e07-1d08083ba220


Currently training NCART model with dataset key: IX_400_pcc95
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 1.281736 |  Val Loss 1.239735
Epoch 1: Train Loss 1.219178 |  Val Loss 1.178427
Epoch 2: Train Loss 1.158600 |  Val Loss 1.119370
Epoch 3: Train Loss 1.100277 |  Val Loss 1.062881
Epoch 4: Train Loss 1.044507 |  Val Loss 1.009243
Epoch 5: Train Loss 0.991583 |  Val Loss 0.958773
Epoch 6: Train Loss 0.941824 |  Val Loss 0.911736
Epoch 7: Train Loss 0.895557 |  Val Loss 0.868467
Epoch 8: Train Loss 0.853066 |  Val Loss 0.829281
Epoch 9: Train Loss 0.814612 |  Val Loss 0.794383
Epoch 10: Train Loss 0.780429 |  Val Loss 0.763933
Epoch 11: Train Loss 0.750672 |  Val Loss 0.738019
Epoch 12: Train Loss 0.725419 |  Val Loss 0.716609
Epoch 13: Train Loss 0.704641 |  Val Loss 0.699572
Epoch 14: Train Loss 0.688197 |  Val Loss 0.686637
Epoch 15: Train Loss 0.675833 |  Val Loss 0.677445
Epoch 16: Train Loss 0.667185 |  Val Loss 0.671548
Epoch 17: Train Loss 0.661798 |  Val Los

[I 2025-01-04 15:30:56,766] Trial 0 finished with value: 0.13271218656114264 and parameters: {'n_trees': 104, 'n_selected': 24, 'n_layers': 1, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.13271218656114264.


Epoch 27: Train Loss 0.671854 |  Val Loss 0.677778
Epoch 28: Train Loss 0.671148 |  Val Loss 0.676226
Epoch 29: Train Loss 0.669666 |  Val Loss 0.674024
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.778133 |  Val Loss 0.752737
Epoch 1: Train Loss 0.750660 |  Val Loss 0.736299
Epoch 2: Train Loss 0.728510 |  Val Loss 0.718698
Epoch 3: Train Loss 0.711765 |  Val Loss 0.707980
Epoch 4: Train Loss 0.695593 |  Val Loss 0.698492
Epoch 5: Train Loss 0.681915 |  Val Loss 0.692188
Epoch 6: Train Loss 0.671461 |  Val Loss 0.682580
Epoch 7: Train Loss 0.661735 |  Val Loss 0.675168
Epoch 8: Train Loss 0.652758 |  Val Loss 0.668599
Epoch 9: Train Loss 0.643369 |  Val Loss 0.659833
Epoch 10: Train Loss 0.634477 |  Val Loss 0.652301
Epoch 11: Train Loss 0.626264 |  Val Loss 0.647245
Epoch 12: Train Loss 0.618783 |  Val Loss 0.640822
Epoch 13: Train Loss 0.612282 |  Val Loss 0.634097
Epoch 14: Train Loss 0.606190 |

[I 2025-01-04 15:30:58,983] Trial 1 finished with value: 0.6251954041004442 and parameters: {'n_trees': 8, 'n_selected': 16, 'n_layers': 8, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.6251954041004442.


...

Epoch 50: Train Loss 0.070839 |  Val Loss 0.484812
Epoch 51: Train Loss 0.066643 |  Val Loss 0.513700
Epoch 52: Train Loss 0.063252 |  Val Loss 0.499885
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.735708 |  Val Loss 0.722009
Epoch 1: Train Loss 0.721242 |  Val Loss 0.710772
Epoch 2: Train Loss 0.709188 |  Val Loss 0.699768
Epoch 3: Train Loss 0.698291 |  Val Loss 0.690263
Epoch 4: Train Loss 0.688271 |  Val Loss 0.681434
Epoch 5: Train Loss 0.678843 |  Val Loss 0.673652
Epoch 6: Train Loss 0.670144 |  Val Loss 0.666775
Epoch 7: Train Loss 0.661882 |  Val Loss 0.659790
Epoch 8: Train Loss 0.654214 |  Val Loss 0.653802
Epoch 9: Train Loss 0.646565 |  Val Loss 0.648853
Epoch 10: Train Loss 0.639428 |  Val Loss 0.644567
Epoch 11: Train Loss 0.632816 |  Val Loss 0.640764
Epoch 12: Train Loss 0.626171 |  Val Loss 0.637376
Epoch 13: Train Loss 0.619700 |  Val Loss 0.635306
Epoch 14: Train Loss 0.613513 |

[I 2025-01-04 15:31:44,737] Trial 13 finished with value: 0.5751797717724086 and parameters: {'n_trees': 128, 'n_selected': 12, 'n_layers': 6, 'mask_type': 'entmax'}. Best is trial 7 with value: 0.6769064842612652.


Epoch 70: Train Loss 0.256843 |  Val Loss 0.509692
Epoch 71: Train Loss 0.253758 |  Val Loss 0.520047
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.720547 |  Val Loss 0.695647
Epoch 1: Train Loss 0.675742 |  Val Loss 0.662312
Epoch 2: Train Loss 0.646462 |  Val Loss 0.639096
Epoch 3: Train Loss 0.626553 |  Val Loss 0.621172
Epoch 4: Train Loss 0.612952 |  Val Loss 0.608797
Epoch 5: Train Loss 0.600134 |  Val Loss 0.598949
Epoch 6: Train Loss 0.587332 |  Val Loss 0.589682
Epoch 7: Train Loss 0.574229 |  Val Loss 0.581014
Epoch 8: Train Loss 0.561021 |  Val Loss 0.573428
Epoch 9: Train Loss 0.546651 |  Val Loss 0.567606
Epoch 10: Train Loss 0.531722 |  Val Loss 0.563078
Epoch 11: Train Loss 0.516136 |  Val Loss 0.558033
Epoch 12: Train Loss 0.500543 |  Val Loss 0.552185
Epoch 13: Train Loss 0.484466 |  Val Loss 0.547395
Epoch 14: Train Loss 0.468220 |  Val Loss 0.543772
Epoch 15: Train Loss 0.452120 |

[I 2025-01-04 15:31:45,879] Trial 14 finished with value: 0.65 and parameters: {'n_trees': 64, 'n_selected': 24, 'n_layers': 8, 'mask_type': 'entmax'}. Best is trial 7 with value: 0.6769064842612652.


Epoch 34: Train Loss 0.171373 |  Val Loss 0.520142
Epoch 35: Train Loss 0.162350 |  Val Loss 0.518972
Epoch 36: Train Loss 0.151643 |  Val Loss 0.530802
Epoch 37: Train Loss 0.143825 |  Val Loss 0.545887
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 72, 'n_selected': 16, 'n_layers': 7, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.708535 |  Val Loss 0.685474
Epoch 1: Train Loss 0.687334 |  Val Loss 0.671899
Epoch 2: Train Loss 0.671932 |  Val Loss 0.659519
Epoch 3: Train Loss 0.659057 |  Val Loss 0.648696
Epoch 4: Train Loss 0.648176 |  Val Loss 0.640351
Epoch 5: Train Loss 0.638432 |  Val Loss 0.632680
Epoch 6: Train Loss 0.629528 |  Val Loss 0.625027
Epoch 7: Train Loss 0.620849 |  Val Loss 0.617836
Epoch 8: Train Loss 0.612410 |  Val Loss 0.610026
Epoch 9: Train Loss 0.603654 |  Val Loss 0.601984
Epoch 10: Train Loss 0.594325 |  Val Loss 0.593652
Epoch 11: Train Loss 0.584623 |  Val Lo

In [31]:
start_time = time.time()
IX_400_pcc75_results_df = train_with_best_hyperparameters(datasets, 'IX_400_pcc75')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

[I 2025-01-04 15:32:00,583] A new study created in memory with name: no-name-b9b39e54-d514-421e-b992-55f60e7aa60e


Currently training NCART model with dataset key: IX_400_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.714669 |  Val Loss 0.689345
Epoch 1: Train Loss 0.695318 |  Val Loss 0.680724
Epoch 2: Train Loss 0.686963 |  Val Loss 0.680150
Epoch 3: Train Loss 0.686470 |  Val Loss 0.681745
Epoch 4: Train Loss 0.687914 |  Val Loss 0.681216
Epoch 5: Train Loss 0.687028 |  Val Loss 0.678079
Epoch 6: Train Loss 0.683413 |  Val Loss 0.673763
Epoch 7: Train Loss 0.678494 |  Val Loss 0.669784
Epoch 8: Train Loss 0.673814 |  Val Loss 0.667033
Epoch 9: Train Loss 0.670323 |  Val Loss 0.665668
Epoch 10: Train Loss 0.668218 |  Val Loss 0.665106
Epoch 11: Train Loss 0.666996 |  Val Loss 0.664508
Epoch 12: Train Loss 0.665824 |  Val Loss 0.663219
Epoch 13: Train Loss 0.664065 |  Val Loss 0.661055
Epoch 14: Train Loss 0.661565 |  Val Loss 0.658309
Epoch 15: Train Loss 0.658611 |  Val Loss 0.655476
Epoch 16: Train Loss 0.655666 |  Val Loss 0.653006
Epoch 17: Train Loss 0.653093 |  Val Los

[I 2025-01-04 15:32:04,091] Trial 0 finished with value: 0.42513287478830203 and parameters: {'n_trees': 24, 'n_selected': 28, 'n_layers': 1, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.42513287478830203.


Epoch 264: Train Loss 0.471644 |  Val Loss 0.558974
Epoch 265: Train Loss 0.471026 |  Val Loss 0.558939
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.703310 |  Val Loss 0.685131
Epoch 1: Train Loss 0.694895 |  Val Loss 0.679256
Epoch 2: Train Loss 0.687769 |  Val Loss 0.673601
Epoch 3: Train Loss 0.681556 |  Val Loss 0.667942
Epoch 4: Train Loss 0.675363 |  Val Loss 0.662751
Epoch 5: Train Loss 0.669216 |  Val Loss 0.657629
Epoch 6: Train Loss 0.663105 |  Val Loss 0.652106
Epoch 7: Train Loss 0.657062 |  Val Loss 0.646591
Epoch 8: Train Loss 0.650844 |  Val Loss 0.641655
Epoch 9: Train Loss 0.644226 |  Val Loss 0.636404
Epoch 10: Train Loss 0.637420 |  Val Loss 0.632095
Epoch 11: Train Loss 0.630872 |  Val Loss 0.628281
Epoch 12: Train Loss 0.624501 |  Val Loss 0.624451
Epoch 13: Train Loss 0.618297 |  Val Loss 0.620828
Epoch 14: Train Loss 0.611948 |  Val Loss 0.616900
Epoch 15: Train Loss 0.605445

[I 2025-01-04 15:32:05,918] Trial 1 finished with value: 0.6752110364284797 and parameters: {'n_trees': 64, 'n_selected': 12, 'n_layers': 6, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.6752110364284797.


...

Epoch 108: Train Loss 0.124553 |  Val Loss 0.390554
Epoch 109: Train Loss 0.121223 |  Val Loss 0.397409
Epoch 110: Train Loss 0.119399 |  Val Loss 0.411126
Epoch 111: Train Loss 0.116846 |  Val Loss 0.413540
Epoch 112: Train Loss 0.113839 |  Val Loss 0.411220
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.700619 |  Val Loss 0.698770
Epoch 1: Train Loss 0.698834 |  Val Loss 0.697228
Epoch 2: Train Loss 0.697183 |  Val Loss 0.695861
Epoch 3: Train Loss 0.695634 |  Val Loss 0.694521
Epoch 4: Train Loss 0.694161 |  Val Loss 0.693328
Epoch 5: Train Loss 0.692791 |  Val Loss 0.692156
Epoch 6: Train Loss 0.691456 |  Val Loss 0.691091
Epoch 7: Train Loss 0.690149 |  Val Loss 0.690089
Epoch 8: Train Loss 0.688851 |  Val Loss 0.689130
Epoch 9: Train Loss 0.687655 |  Val Loss 0.688072
Epoch 10: Train Loss 0.686514 |  Val Loss 0.687047
Epoch 11: Train Loss 0.685415 |  Val Loss 0.686088
Epoch 12: Train Loss 0.684

[I 2025-01-04 15:32:34,592] Trial 13 finished with value: 0.629940788348712 and parameters: {'n_trees': 88, 'n_selected': 4, 'n_layers': 3, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.6752110364284797.


Epoch 195: Train Loss 0.307405 |  Val Loss 0.493081
Epoch 196: Train Loss 0.305412 |  Val Loss 0.498098
Epoch 197: Train Loss 0.302821 |  Val Loss 0.502560
Epoch 198: Train Loss 0.299963 |  Val Loss 0.503666
Epoch 199: Train Loss 0.298121 |  Val Loss 0.500773
Epoch 200: Train Loss 0.295676 |  Val Loss 0.495669
Epoch 201: Train Loss 0.292794 |  Val Loss 0.495108
Epoch 202: Train Loss 0.290865 |  Val Loss 0.496832
Epoch 203: Train Loss 0.288732 |  Val Loss 0.502190
Epoch 204: Train Loss 0.286214 |  Val Loss 0.505226
Epoch 205: Train Loss 0.283994 |  Val Loss 0.503332
Epoch 206: Train Loss 0.281717 |  Val Loss 0.498608
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.702266 |  Val Loss 0.698819
Epoch 1: Train Loss 0.695479 |  Val Loss 0.695572
Epoch 2: Train Loss 0.689804 |  Val Loss 0.692380
Epoch 3: Train Loss 0.684688 |  Val Loss 0.688738
Epoch 4: Train Loss 0.679914 |  Val Loss 0.686171
Epoch 5: Train

[I 2025-01-04 15:32:37,282] Trial 14 finished with value: 0.6251954041004442 and parameters: {'n_trees': 32, 'n_selected': 12, 'n_layers': 6, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.6752110364284797.


Epoch 111: Train Loss 0.126615 |  Val Loss 0.491811
Epoch 112: Train Loss 0.124440 |  Val Loss 0.490471
Epoch 113: Train Loss 0.121579 |  Val Loss 0.489984
Epoch 114: Train Loss 0.118880 |  Val Loss 0.497057
Epoch 115: Train Loss 0.116473 |  Val Loss 0.498812
Epoch 116: Train Loss 0.113812 |  Val Loss 0.497660
Epoch 117: Train Loss 0.111456 |  Val Loss 0.498777
Epoch 118: Train Loss 0.109054 |  Val Loss 0.501595
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 64, 'n_selected': 12, 'n_layers': 6, 'mask_type': 'sparsemax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.703310 |  Val Loss 0.685131
Epoch 1: Train Loss 0.694895 |  Val Loss 0.679256
Epoch 2: Train Loss 0.687769 |  Val Loss 0.673601
Epoch 3: Train Loss 0.681556 |  Val Loss 0.667942
Epoch 4: Train Loss 0.675363 |  Val Loss 0.662751
Epoch 5: Train Loss 0.669216 |  Val Loss 0.657629
Epoch 6: Train Loss 0.663105 |  Val Loss 0.652106
Epoch 7: Train Loss 0.657

In [32]:
start_time = time.time()
IX_300_nopcc_results_df = train_with_best_hyperparameters(datasets, 'IX_300_nopcc')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

[I 2025-01-04 15:32:53,129] A new study created in memory with name: no-name-3b31008a-fcc7-4293-81e9-e2543c4ef62b


Currently training NCART model with dataset key: IX_300_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.711946 |  Val Loss 0.704858
Epoch 1: Train Loss 0.703108 |  Val Loss 0.698380
Epoch 2: Train Loss 0.695587 |  Val Loss 0.691063
Epoch 3: Train Loss 0.688855 |  Val Loss 0.684417
Epoch 4: Train Loss 0.682471 |  Val Loss 0.678532
Epoch 5: Train Loss 0.676599 |  Val Loss 0.672466
Epoch 6: Train Loss 0.670996 |  Val Loss 0.667164
Epoch 7: Train Loss 0.665473 |  Val Loss 0.662567
Epoch 8: Train Loss 0.660062 |  Val Loss 0.658230
Epoch 9: Train Loss 0.654998 |  Val Loss 0.654050
Epoch 10: Train Loss 0.650068 |  Val Loss 0.649566
Epoch 11: Train Loss 0.645413 |  Val Loss 0.645762
Epoch 12: Train Loss 0.640905 |  Val Loss 0.642412
Epoch 13: Train Loss 0.636646 |  Val Loss 0.639192
Epoch 14: Train Loss 0.632594 |  Val Loss 0.635930
Epoch 15: Train Loss 0.628628 |  Val Loss 0.632982
Epoch 16: Train Loss 0.624783 |  Val Loss 0.630179
Epoch 17: Train Loss 0.620911 |  Val Los

[I 2025-01-04 15:32:54,856] Trial 0 finished with value: 0.6365239496641676 and parameters: {'n_trees': 104, 'n_selected': 8, 'n_layers': 4, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.6365239496641676.


Epoch 97: Train Loss 0.231344 |  Val Loss 0.445202
Epoch 98: Train Loss 0.227504 |  Val Loss 0.451935
Epoch 99: Train Loss 0.224219 |  Val Loss 0.447266
Epoch 100: Train Loss 0.220513 |  Val Loss 0.443472
Epoch 101: Train Loss 0.217155 |  Val Loss 0.437167
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.732956 |  Val Loss 0.696799
Epoch 1: Train Loss 0.691463 |  Val Loss 0.665087
Epoch 2: Train Loss 0.662074 |  Val Loss 0.643313
Epoch 3: Train Loss 0.638403 |  Val Loss 0.621647
Epoch 4: Train Loss 0.617982 |  Val Loss 0.605998
Epoch 5: Train Loss 0.601188 |  Val Loss 0.599506
Epoch 6: Train Loss 0.585832 |  Val Loss 0.590100
Epoch 7: Train Loss 0.572106 |  Val Loss 0.581981
Epoch 8: Train Loss 0.558239 |  Val Loss 0.575281
Epoch 9: Train Loss 0.545700 |  Val Loss 0.568650
Epoch 10: Train Loss 0.532264 |  Val Loss 0.561790
Epoch 11: Train Loss 0.517843 |  Val Loss 0.554008
Epoch 12: Train Loss 0.502163

[I 2025-01-04 15:32:56,211] Trial 1 finished with value: 0.6423172335936725 and parameters: {'n_trees': 112, 'n_selected': 20, 'n_layers': 7, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.6423172335936725.


...

Epoch 341: Train Loss 0.121336 |  Val Loss 0.433989
Epoch 342: Train Loss 0.120745 |  Val Loss 0.433843
Epoch 343: Train Loss 0.120015 |  Val Loss 0.433203
Epoch 344: Train Loss 0.119082 |  Val Loss 0.433095
Epoch 345: Train Loss 0.118046 |  Val Loss 0.432887
Epoch 346: Train Loss 0.117025 |  Val Loss 0.432759
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.838529 |  Val Loss 0.797638
Epoch 1: Train Loss 0.812564 |  Val Loss 0.778583
Epoch 2: Train Loss 0.791103 |  Val Loss 0.761806
Epoch 3: Train Loss 0.771998 |  Val Loss 0.746488
Epoch 4: Train Loss 0.754112 |  Val Loss 0.734121
Epoch 5: Train Loss 0.737073 |  Val Loss 0.721688
Epoch 6: Train Loss 0.721292 |  Val Loss 0.709436
Epoch 7: Train Loss 0.706375 |  Val Loss 0.698161
Epoch 8: Train Loss 0.691870 |  Val Loss 0.686728
Epoch 9: Train Loss 0.677858 |  Val Loss 0.674710
Epoch 10: Train Loss 0.664695 |  Val Loss 0.663223
Epoch 11: Train Loss 0.65

[I 2025-01-04 15:33:30,169] Trial 13 finished with value: 0.6681531047810609 and parameters: {'n_trees': 64, 'n_selected': 12, 'n_layers': 5, 'mask_type': 'entmax'}. Best is trial 6 with value: 0.7670929478598306.


Epoch 82: Train Loss 0.150982 |  Val Loss 0.428867
Epoch 83: Train Loss 0.148123 |  Val Loss 0.430049
Epoch 84: Train Loss 0.144390 |  Val Loss 0.443471
Epoch 85: Train Loss 0.141282 |  Val Loss 0.465776
Epoch 86: Train Loss 0.138566 |  Val Loss 0.488730
Epoch 87: Train Loss 0.135522 |  Val Loss 0.470638
Epoch 88: Train Loss 0.132897 |  Val Loss 0.467919
Epoch 89: Train Loss 0.129702 |  Val Loss 0.459530
Epoch 90: Train Loss 0.127276 |  Val Loss 0.443296
Epoch 91: Train Loss 0.124677 |  Val Loss 0.451769
Epoch 92: Train Loss 0.122013 |  Val Loss 0.468517
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.795839 |  Val Loss 0.757693
Epoch 1: Train Loss 0.781169 |  Val Loss 0.746717
Epoch 2: Train Loss 0.767808 |  Val Loss 0.735389
Epoch 3: Train Loss 0.755461 |  Val Loss 0.724932
Epoch 4: Train Loss 0.744092 |  Val Loss 0.715116
Epoch 5: Train Loss 0.733696 |  Val Loss 0.705390
Epoch 6: Train Loss 0.72368

[I 2025-01-04 15:33:32,408] Trial 14 finished with value: 0.565685424949238 and parameters: {'n_trees': 128, 'n_selected': 12, 'n_layers': 3, 'mask_type': 'entmax'}. Best is trial 6 with value: 0.7670929478598306.


Epoch 124: Train Loss 0.160762 |  Val Loss 0.491112
Epoch 125: Train Loss 0.159166 |  Val Loss 0.494113
Epoch 126: Train Loss 0.154388 |  Val Loss 0.483242
Epoch 127: Train Loss 0.153221 |  Val Loss 0.475743
Epoch 128: Train Loss 0.149141 |  Val Loss 0.486906
Epoch 129: Train Loss 0.146497 |  Val Loss 0.483425
Epoch 130: Train Loss 0.142817 |  Val Loss 0.473817
Epoch 131: Train Loss 0.140589 |  Val Loss 0.508377
Epoch 132: Train Loss 0.136491 |  Val Loss 0.520131
Epoch 133: Train Loss 0.134669 |  Val Loss 0.505555
Epoch 134: Train Loss 0.131666 |  Val Loss 0.499313
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 96, 'n_selected': 4, 'n_layers': 2, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.707070 |  Val Loss 0.708875
Epoch 1: Train Loss 0.705256 |  Val Loss 0.707164
Epoch 2: Train Loss 0.703389 |  Val Loss 0.705743
Epoch 3: Train Loss 0.701919 |  Val Loss 0.704060
Epoch 4: Train Loss 0.7

In [33]:
start_time = time.time()
IX_300_pcc95_results_df = train_with_best_hyperparameters(datasets, 'IX_300_pcc95')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

[I 2025-01-04 15:34:25,269] A new study created in memory with name: no-name-1993f9c1-6231-44fb-9413-3374f6f4583c


Currently training NCART model with dataset key: IX_300_pcc95
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.996269 |  Val Loss 0.880504
Epoch 1: Train Loss 0.919507 |  Val Loss 0.812347
Epoch 2: Train Loss 0.853278 |  Val Loss 0.755302
Epoch 3: Train Loss 0.794766 |  Val Loss 0.706887
Epoch 4: Train Loss 0.743871 |  Val Loss 0.664195
Epoch 5: Train Loss 0.700769 |  Val Loss 0.629755
Epoch 6: Train Loss 0.667012 |  Val Loss 0.604414
Epoch 7: Train Loss 0.640992 |  Val Loss 0.585933
Epoch 8: Train Loss 0.622282 |  Val Loss 0.574745
Epoch 9: Train Loss 0.610621 |  Val Loss 0.568773
Epoch 10: Train Loss 0.603964 |  Val Loss 0.565862
Epoch 11: Train Loss 0.600936 |  Val Loss 0.565585
Epoch 12: Train Loss 0.599842 |  Val Loss 0.566538
Epoch 13: Train Loss 0.599294 |  Val Loss 0.567539
Epoch 14: Train Loss 0.598642 |  Val Loss 0.568032
Epoch 15: Train Loss 0.597001 |  Val Loss 0.567354
Epoch 16: Train Loss 0.593778 |  Val Loss 0.565316
Epoch 17: Train Loss 0.589063 |  Val Los

[I 2025-01-04 15:34:26,426] Trial 0 finished with value: 0.6365239496641676 and parameters: {'n_trees': 48, 'n_selected': 28, 'n_layers': 3, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.6365239496641676.


Epoch 56: Train Loss 0.330973 |  Val Loss 0.483759
Epoch 57: Train Loss 0.323887 |  Val Loss 0.484778
Epoch 58: Train Loss 0.316643 |  Val Loss 0.483971
Epoch 59: Train Loss 0.309123 |  Val Loss 0.480288
Epoch 60: Train Loss 0.302254 |  Val Loss 0.482174
Epoch 61: Train Loss 0.294477 |  Val Loss 0.482275
Epoch 62: Train Loss 0.287070 |  Val Loss 0.486102
Epoch 63: Train Loss 0.279782 |  Val Loss 0.489072
Epoch 64: Train Loss 0.272503 |  Val Loss 0.490185
Epoch 65: Train Loss 0.265304 |  Val Loss 0.489698
Epoch 66: Train Loss 0.257603 |  Val Loss 0.487971
Epoch 67: Train Loss 0.250145 |  Val Loss 0.488391
Epoch 68: Train Loss 0.242811 |  Val Loss 0.491589
Epoch 69: Train Loss 0.235430 |  Val Loss 0.497618
Epoch 70: Train Loss 0.228021 |  Val Loss 0.501834
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.720408 |  Val Loss 0.732698
Epoch 1: Train Loss 0.690910 |  Val Loss 0.717507
Epoch 2: Train Loss 0.6

[I 2025-01-04 15:34:27,525] Trial 1 finished with value: 0.6013377943029549 and parameters: {'n_trees': 56, 'n_selected': 28, 'n_layers': 3, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.6365239496641676.


...

Epoch 162: Train Loss 0.306545 |  Val Loss 0.535947
Epoch 163: Train Loss 0.303964 |  Val Loss 0.540120
Epoch 164: Train Loss 0.301807 |  Val Loss 0.531961
Epoch 165: Train Loss 0.299456 |  Val Loss 0.535395
Epoch 166: Train Loss 0.296979 |  Val Loss 0.539087
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.804995 |  Val Loss 0.780556
Epoch 1: Train Loss 0.749299 |  Val Loss 0.735593
Epoch 2: Train Loss 0.707510 |  Val Loss 0.698325
Epoch 3: Train Loss 0.673186 |  Val Loss 0.668825
Epoch 4: Train Loss 0.644634 |  Val Loss 0.645423
Epoch 5: Train Loss 0.621518 |  Val Loss 0.627885
Epoch 6: Train Loss 0.603003 |  Val Loss 0.615073
Epoch 7: Train Loss 0.588474 |  Val Loss 0.605879
Epoch 8: Train Loss 0.576897 |  Val Loss 0.599904
Epoch 9: Train Loss 0.567260 |  Val Loss 0.595738
Epoch 10: Train Loss 0.558455 |  Val Loss 0.591989
Epoch 11: Train Loss 0.549829 |  Val Loss 0.587580
Epoch 12: Train Loss 0.540

[I 2025-01-04 15:35:15,387] Trial 13 finished with value: 0.7670929478598306 and parameters: {'n_trees': 88, 'n_selected': 20, 'n_layers': 5, 'mask_type': 'sparsemax'}. Best is trial 4 with value: 0.8.


Epoch 44: Train Loss 0.158852 |  Val Loss 0.479856
Epoch 45: Train Loss 0.150917 |  Val Loss 0.471578
Epoch 46: Train Loss 0.143250 |  Val Loss 0.466290
Epoch 47: Train Loss 0.136380 |  Val Loss 0.467757
Epoch 48: Train Loss 0.129981 |  Val Loss 0.458258
Epoch 49: Train Loss 0.123684 |  Val Loss 0.473336
Epoch 50: Train Loss 0.117655 |  Val Loss 0.483284
Epoch 51: Train Loss 0.112146 |  Val Loss 0.465979
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.700657 |  Val Loss 0.709697
Epoch 1: Train Loss 0.692021 |  Val Loss 0.708564
Epoch 2: Train Loss 0.685654 |  Val Loss 0.703183
Epoch 3: Train Loss 0.680319 |  Val Loss 0.697084
Epoch 4: Train Loss 0.674044 |  Val Loss 0.689852
Epoch 5: Train Loss 0.668159 |  Val Loss 0.680844
Epoch 6: Train Loss 0.662725 |  Val Loss 0.674838
Epoch 7: Train Loss 0.657579 |  Val Loss 0.670604
Epoch 8: Train Loss 0.652926 |  Val Loss 0.667193
Epoch 9: Train Loss 0.648533 |

[I 2025-01-04 15:35:17,975] Trial 14 finished with value: 0.6726727939963124 and parameters: {'n_trees': 96, 'n_selected': 20, 'n_layers': 2, 'mask_type': 'sparsemax'}. Best is trial 4 with value: 0.8.


Streaming output truncated to the last 5000 lines.
Epoch 279: Train Loss 0.520541 |  Val Loss 0.609892
Epoch 280: Train Loss 0.520097 |  Val Loss 0.609814
Epoch 281: Train Loss 0.519657 |  Val Loss 0.609739
Epoch 282: Train Loss 0.519219 |  Val Loss 0.609664
Epoch 283: Train Loss 0.518781 |  Val Loss 0.609586
Epoch 284: Train Loss 0.518342 |  Val Loss 0.609506
Epoch 285: Train Loss 0.517904 |  Val Loss 0.609433
Epoch 286: Train Loss 0.517467 |  Val Loss 0.609357
Epoch 287: Train Loss 0.517028 |  Val Loss 0.609283
Epoch 288: Train Loss 0.516591 |  Val Loss 0.609211
Epoch 289: Train Loss 0.516155 |  Val Loss 0.609139
Epoch 290: Train Loss 0.515719 |  Val Loss 0.609068
Epoch 291: Train Loss 0.515284 |  Val Loss 0.609001
Epoch 292: Train Loss 0.514849 |  Val Loss 0.608934
Epoch 293: Train Loss 0.514417 |  Val Loss 0.608872
Epoch 294: Train Loss 0.513986 |  Val Loss 0.608807
Epoch 295: Train Loss 0.513555 |  Val Loss 0.608736
Epoch 296: Train Loss 0.513124 |  Val Loss 0.608666
Epoch 297: Tr

In [34]:
start_time = time.time()
IX_300_pcc75_results_df = train_with_best_hyperparameters(datasets, 'IX_300_pcc75')
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print("\n---------------------------------------------------------------------\n")

[I 2025-01-04 15:36:50,859] A new study created in memory with name: no-name-f3b93a5a-a854-43e9-ab56-a7eedad684ae


Currently training NCART model with dataset key: IX_300_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.948800 |  Val Loss 0.882707
Epoch 1: Train Loss 0.893596 |  Val Loss 0.834321
Epoch 2: Train Loss 0.843606 |  Val Loss 0.791649
Epoch 3: Train Loss 0.797341 |  Val Loss 0.753444
Epoch 4: Train Loss 0.755716 |  Val Loss 0.718447
Epoch 5: Train Loss 0.718891 |  Val Loss 0.687097
Epoch 6: Train Loss 0.687597 |  Val Loss 0.660255
Epoch 7: Train Loss 0.660979 |  Val Loss 0.637843
Epoch 8: Train Loss 0.639003 |  Val Loss 0.620173
Epoch 9: Train Loss 0.621294 |  Val Loss 0.606207
Epoch 10: Train Loss 0.607256 |  Val Loss 0.595526
Epoch 11: Train Loss 0.596266 |  Val Loss 0.587528
Epoch 12: Train Loss 0.587799 |  Val Loss 0.581756
Epoch 13: Train Loss 0.581231 |  Val Loss 0.577293
Epoch 14: Train Loss 0.575533 |  Val Loss 0.574281
Epoch 15: Train Loss 0.570518 |  Val Loss 0.572020
Epoch 16: Train Loss 0.565461 |  Val Loss 0.569785
Epoch 17: Train Loss 0.560220 |  Val Los

[I 2025-01-04 15:36:51,854] Trial 0 finished with value: 0.6013377943029549 and parameters: {'n_trees': 96, 'n_selected': 24, 'n_layers': 6, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.6013377943029549.


Epoch 48: Train Loss 0.228592 |  Val Loss 0.494781
Epoch 49: Train Loss 0.217633 |  Val Loss 0.495714
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.839879 |  Val Loss 0.787878
Epoch 1: Train Loss 0.785991 |  Val Loss 0.741217
Epoch 2: Train Loss 0.740759 |  Val Loss 0.702140
Epoch 3: Train Loss 0.702692 |  Val Loss 0.672560
Epoch 4: Train Loss 0.672733 |  Val Loss 0.649943
Epoch 5: Train Loss 0.649891 |  Val Loss 0.635302
Epoch 6: Train Loss 0.634097 |  Val Loss 0.626435
Epoch 7: Train Loss 0.624567 |  Val Loss 0.621936
Epoch 8: Train Loss 0.619059 |  Val Loss 0.618918
Epoch 9: Train Loss 0.616123 |  Val Loss 0.616772
Epoch 10: Train Loss 0.613742 |  Val Loss 0.613799
Epoch 11: Train Loss 0.610526 |  Val Loss 0.609972
Epoch 12: Train Loss 0.606389 |  Val Loss 0.605425
Epoch 13: Train Loss 0.601102 |  Val Loss 0.599252
Epoch 14: Train Loss 0.593869 |  Val Loss 0.591707
Epoch 15: Train Loss 0.585153 |

[I 2025-01-04 15:36:52,980] Trial 1 finished with value: 0.5695214286468868 and parameters: {'n_trees': 104, 'n_selected': 28, 'n_layers': 4, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.6013377943029549.


...

Epoch 247: Train Loss 0.106116 |  Val Loss 0.426795
Epoch 248: Train Loss 0.104426 |  Val Loss 0.425655
Epoch 249: Train Loss 0.102841 |  Val Loss 0.426725
Epoch 250: Train Loss 0.101259 |  Val Loss 0.428619
Epoch 251: Train Loss 0.099734 |  Val Loss 0.427353
Epoch 252: Train Loss 0.098180 |  Val Loss 0.426397
Epoch 253: Train Loss 0.096701 |  Val Loss 0.427186
Epoch 254: Train Loss 0.095234 |  Val Loss 0.427597
Epoch 255: Train Loss 0.093820 |  Val Loss 0.428557
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.700861 |  Val Loss 0.698170
Epoch 1: Train Loss 0.699235 |  Val Loss 0.696724
Epoch 2: Train Loss 0.697696 |  Val Loss 0.695371
Epoch 3: Train Loss 0.696250 |  Val Loss 0.694102
Epoch 4: Train Loss 0.694893 |  Val Loss 0.692923
Epoch 5: Train Loss 0.693624 |  Val Loss 0.691836
Epoch 6: Train Loss 0.692445 |  Val Loss 0.690837
Epoch 7: Train Loss 0.691357 |  Val Loss 0.689924
Epoch 8: Train Loss 

[I 2025-01-04 15:37:47,382] Trial 13 finished with value: 0.6666666666666666 and parameters: {'n_trees': 48, 'n_selected': 8, 'n_layers': 1, 'mask_type': 'entmax'}. Best is trial 3 with value: 0.734968415259167.


Epoch 991: Train Loss 0.294591 |  Val Loss 0.471335
Epoch 992: Train Loss 0.294171 |  Val Loss 0.471181
Epoch 993: Train Loss 0.293750 |  Val Loss 0.471020
Epoch 994: Train Loss 0.293330 |  Val Loss 0.470870
Epoch 995: Train Loss 0.292911 |  Val Loss 0.470714
Epoch 996: Train Loss 0.292490 |  Val Loss 0.470543
Epoch 997: Train Loss 0.292067 |  Val Loss 0.470361
Epoch 998: Train Loss 0.291648 |  Val Loss 0.470201
Epoch 999: Train Loss 0.291227 |  Val Loss 0.470060
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.748941 |  Val Loss 0.678358
Epoch 1: Train Loss 0.693945 |  Val Loss 0.645580
Epoch 2: Train Loss 0.662503 |  Val Loss 0.632302
Epoch 3: Train Loss 0.644941 |  Val Loss 0.624076
Epoch 4: Train Loss 0.634351 |  Val Loss 0.621054
Epoch 5: Train Loss 0.626178 |  Val Loss 0.617227
Epoch 6: Train Loss 0.617621 |  Val Loss 0.609809
Epoch 7: Train Loss 0.606733 |  Val Loss 0.598457
Epoch 8: Train Loss 0.593614 |  Val Loss 0.588796
Epoch 9: Train Loss 0.579812 |  Val Loss 

[I 2025-01-04 15:37:48,880] Trial 14 finished with value: 0.6666666666666666 and parameters: {'n_trees': 80, 'n_selected': 32, 'n_layers': 6, 'mask_type': 'sparsemax'}. Best is trial 3 with value: 0.734968415259167.


Streaming output truncated to the last 5000 lines.
Epoch 448: Train Loss 0.399930 |  Val Loss 0.471989
Epoch 449: Train Loss 0.399388 |  Val Loss 0.471776
Epoch 450: Train Loss 0.398842 |  Val Loss 0.471557
Epoch 451: Train Loss 0.398297 |  Val Loss 0.471334
Epoch 452: Train Loss 0.397751 |  Val Loss 0.471108
Epoch 453: Train Loss 0.397205 |  Val Loss 0.470885
Epoch 454: Train Loss 0.396659 |  Val Loss 0.470657
Epoch 455: Train Loss 0.396115 |  Val Loss 0.470430
Epoch 456: Train Loss 0.395572 |  Val Loss 0.470206
Epoch 457: Train Loss 0.395030 |  Val Loss 0.469981
Epoch 458: Train Loss 0.394488 |  Val Loss 0.469758
Epoch 459: Train Loss 0.393944 |  Val Loss 0.469537
Epoch 460: Train Loss 0.393402 |  Val Loss 0.469317
Epoch 461: Train Loss 0.392860 |  Val Loss 0.469095
Epoch 462: Train Loss 0.392317 |  Val Loss 0.468866
Epoch 463: Train Loss 0.391773 |  Val Loss 0.468636
Epoch 464: Train Loss 0.391230 |  Val Loss 0.468411
Epoch 465: Train Loss 0.390686 |  Val Loss 0.468190
Epoch 466: Tr

In [35]:
print("Base Model:")
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_nopcc_results_df, 'IX_all_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_pcc95_results_df, 'IX_all_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_nopcc_results_df, 'IX_all_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_nopcc_results_df, 'IX_500_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_pcc95_results_df, 'IX_500_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_nopcc_results_df, 'IX_500_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_nopcc_results_df, 'IX_400_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_pcc95_results_df, 'IX_400_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_nopcc_results_df, 'IX_400_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_nopcc_results_df, 'IX_300_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_pcc95_results_df, 'IX_300_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_nopcc_results_df, 'IX_300_pcc75')
print('\n---------------------------------------------------------------------\n')


Base Model:

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: IX_all_nopcc
Fold 0, Accuracy: 0.855, MCC: 0.716
Fold 1, Accuracy: 0.848, MCC: 0.710
Fold 2, Accuracy: 0.883, MCC: 0.766
Fold 3, Accuracy: 0.736, MCC: 0.506
Fold 4, Accuracy: 0.792, MCC: 0.587
Fold 5, Accuracy: 0.799, MCC: 0.611
Fold 6, Accuracy: 0.743, MCC: 0.487
Fold 7, Accuracy: 0.799, MCC: 0.598
Fold 8, Accuracy: 0.764, MCC: 0.530
Fold 9, Accuracy: 0.847, MCC: 0.695

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: IX_all_pcc95
Fold 0, Accuracy: 0.800, MCC: 0.613
Fold 1, Accuracy: 0.862, MCC: 0.724
Fold 2, Accuracy: 0.869, MCC: 0.738
Fold 3, Accuracy: 0.840, MCC: 0.686
Fold 4, Accuracy: 0.792, MCC: 0.607
Fold 5, Accuracy: 0.785, MCC: 0.579
Fold 6, Accuracy: 0.792, MCC: 0.584
Fold 7, Accuracy: 0.806, MCC: 0.611
Fold 8, Accuracy: 0.812, MCC: 0.627
Fold 9, Accuracy: 